In [1]:


using GLM, MixedModels, RCall, DataFrames, Distributions
#Occ
R""" load("/media/u01/analytics/scoring/Campbell_Soup_785/Output/saved_data/DolOcc_final_model_data.Rdata") """

@rget finaldata_dolocc;
df_data=finaldata_dolocc
df_data[:log_PRE_P1] = log(Array(df_data[:Dol_per_Trip_PRE_P1]) + 1);

f1=Dol_per_Trip_POS_P1~ Trps_PRE_P3 + Trps_PRE_P4 + Trps_PRE_P2 + Cpn_Trps_Shr_DPP_P1 + Dis_Trps_Shr_DPP_P1 + Nonbuyer_Pre_P1 + group+(1+group|Publisher_fct)+(1+group|Publisher_fct_1)
# Array(df_data[:log_PRE_P1])  offset(log(Dol_per_Trip_PRE_P1+1))
gm = glm(f1, df_data, Gamma(), LogLink())
gm = glm(f1, df_data, Gamma(), LogLink(), offset = Array(df_data[:log_PRE_P1]))
glmm2 = fit!(glmm(f1, df_data, Gamma(), LogLink()))
# test offset: gm = glm(Dol_per_Trip_POS_P1 ~ 1+Dol_per_Trip_PRE_P1, df_data, Gamma(), LogLink())



pool!(df_data, [:Publisher_fct,:Publisher_fct_1])


#@rget dolocc_Formula_final
#f1=Dol_per_Trip_POS_P1~ Trps_PRE_P3 + Trps_PRE_P4 + Trps_PRE_P2 + Cpn_Trps_Shr_DPP_P1 + Dis_Trps_Shr_DPP_P1 + Nonbuyer_Pre_P1 + offset(log(Dol_per_Trip_PRE_P1+1)) + group+(0+group|Publisher_fct)+(0+group|Publisher_fct_1)
#revised 
f1=Dol_per_Trip_POS_P1~ Trps_PRE_P3 + Trps_PRE_P4 + Trps_PRE_P2 + Cpn_Trps_Shr_DPP_P1 + Dis_Trps_Shr_DPP_P1 + Nonbuyer_Pre_P1 + group+(1+group|Publisher_fct)+(1+group|Publisher_fct_1)
 offset(log(Dol_per_Trip_PRE_P1+1))

f1=Dol_per_Trip_POS_P1 ~ 1 + Trps_PRE_P3 + Trps_PRE_P4 + Trps_PRE_P2 + Cpn_Trps_Shr_DPP_P1 + Dis_Trps_Shr_DPP_P1 + Nonbuyer_Pre_P1 + group+(1+group|Publisher_fct)+(1+group|Publisher_fct_1)
f1=Dol_per_Trip_POS_P1 ~ 1 + Trps_PRE_P3 + Trps_PRE_P4 + Trps_PRE_P2 + Cpn_Trps_Shr_DPP_P1 + Dis_Trps_Shr_DPP_P1 + Nonbuyer_Pre_P1 + group+(1|Publisher_fct)+(1|Publisher_fct_1)


#df_data[:log_PRE_P1] = log(Array(df_data[:Dol_per_Trip_PRE_P1]) + 1);
#gm = glm(f1, df_data, Gamma(), LogLink(), offset = Array(df_data[:log_PRE_P1]))
gm = glm(f1, df_data, Gamma(), LogLink())
glmm2 = fit!(glmm(f1, df_data, Gamma(), LogLink()))

f1 = Dol_per_Trip_POS_P1 ~ 1 + Trps_PRE_P4 + Cpn_UN_per_Trip_DPP_P8 + Dis_Trps_Shr_DPP_P8 + Trps_PRE_P2 + Nonbuyer_Pre_P1 + log_PRE_P1 + (1 | Creative_Groups);
f1 = Dol_per_Trip_POS_P1 ~ 1 + Trps_PRE_P4 + Trps_PRE_P2 + Nonbuyer_Pre_P1 + log_PRE_P1 + (1 | Publisher_fct);
#@time m1 = fit!(glmm(f1, occ_final_data, Poisson()))
#gm7 = glm(Dol_per_Trip_POS_P1 ~ 1 + log_PRE_P1, dolocc,  Gamma(), LogLink())
#gm8 = glm(Dol_per_Trip_POS_P1 ~ 1, dolocc, Gamma(), LogLink(), offset = Array(dolocc[:log_PRE_P1]))
#glmm2 = fit!(glmm(fdoloccmm, dolocc, Gamma(), LogLink()))
#@time fit!(glmm(fdoloccmm, dolocc, Gamma(), LogLink()));
#gm2 = glm(Buyer_Pos_P1 ~ 1, pen, Bernoulli(), offset = log(Array(pen[:Buyer_Pre_P1] + 1)))
#gm8 = glm(Dol_per_Trip_POS_P1 ~ 1, dolocc, Gamma(), LogLink(), offset = Array(dolocc[:log_PRE_P1]))

gm = glm(f1, df_data, Gamma(), LogLink(), offset = Array(df_data[:log_PRE_P1]))
glmmx = fit!(glmm(f1, df_data, Gamma(), LogLink(),offset = Array(df_data[:log_PRE_P1]) ))

glmmx = glmm(f1, df_data, Gamma(), LogLink(), offset = Array(df_data[:log_PRE_P1]))


In [ ]:
using DataStructures, DataFrames, StatsFuns, GLM , JuMP,NLopt, StatStack, HDF5, JLD
cd("/media/u01/analytics/scoring/Healthy/modeling5/")
df_data = readtable("csv_final_healthychoice1_I3.csv",header=true); #lowercase(df_data)
df_h = readtable("Headers_healthy_choice3.csv",header=false); #lowercase(df_data)
names!(df_data, convert(Array{Symbol}, df_h[:x1]) ) 

#save("data.jld", "data", df_data, compress=true)
#load("data.jld")["data"]

exposed_flag_var = :exposed_flag  
pvalue_lvl = 0.20
all_mandatory_vars = [:Buyer_Pos_P1,:Buyer_Pre_P1,:Trps_POS_P1,:Trps_PRE_P1,:Dol_per_Trip_POS_P1,:Dol_per_Trip_PRE_P1,:Nonbuyer_Pre_P1,:hh_age,:estimated_hh_income,:number_of_children_in_living_Un,:person_1_gender,:experian_id]
random_demos  = [:estimated_hh_income,:hh_age,:number_of_children_in_living_Un]
random_campaigns = [:Creative_Groups] # "estimated_hh_income", "hh_age", "number_of_children_in_living_Un"
allrandoms = vcat(random_demos,random_campaigns)
P2_Competitor = true
dropvars = ["XXX"]
#######################################
#--------- Data Manipulation ---------#
#######################################

df_data[:group] = df_data[exposed_flag_var]
df_data[:panid] = df_data[:experian_id]  
ProScore = df_h[:x1][find(x -> contains(string(x),"MODEL") ,df_h[:x1])][1]
ProScore = symbol(ProScore)
df_data[df_data[:number_of_children_in_living_Un].>=4,:number_of_children_in_living_Un] = 4


#pool!(df_data, allrandoms)


In [17]:
# --- some other stuff

In [ ]:

dft = DataFrame(names=names(df_data),eltypes=eltypes(df_data))
for c in dft[dft[:eltypes].==UTF8String,:names]
    println(c)
    map(x -> int(x),df_data[:estimated_hh_income])
end

for c in dft[  (dft[:eltypes].==Int64)|(dft[:eltypes].==Float64)   ,:names]
    println(c)
    df_data[isna(df_data[c]),c] = 0.0  # replace NAs with 0.0
end

#df_data[isna(df_data[:group]),:group] = "0"  # but is an Int64, so already reset NAs
#supposed to factoorize :group, .. but not sure this is required


In [ ]:
#----------------------- here ------------------------------------------------------


#indx <- sapply(finaldata, is.character)
#options(warn=-1)
#finaldata[indx] <- lapply(finaldata[indx],function(x) as.numeric(as.character(x)))
#finaldata[,exposed_flag_var] <- NULL
#options(warn=0)

# replace NAs with zero for numeric variables 
#finaldata <- as.data.frame(finaldata)
#finaldata[sapply(finaldata, is.numeric)][is.na(finaldata[sapply(finaldata, is.numeric)])] <- 0
#finaldata <- as.data.table(finaldata)

# set as zero the //N value of the group variable
#finaldata <- finaldata[is.na(group),group:="0"]
#finaldata <- finaldata[group=="\\N",group:="0"]
#finaldata$group <- factor(finaldata$group)
#finaldata$group <- relevel(finaldata$group,"0")

In [ ]:
###################################### Do Later ##############################################################
########## QCs ##########

# Create a summary of the dataset
t1 <- as.data.frame(finaldata[,sapply(finaldata,is.numeric)])
finaldata_num <- as.data.frame(finaldata)
finaldata_num <- finaldata_num[,c(which(t1==T))]
finaldata_num <- as.data.table(finaldata_num)
finaldata_num <- finaldata_num[,c(3:(which(names(finaldata_num)=="state")-1),((which(names(finaldata_num)=="mail_responder"))+1):length(names(finaldata_num))),with=F]
finaldata_num <- as.data.frame(finaldata_num)

summary_num <- sapply(finaldata_num[,!names(finaldata_num) %in% c("exposed_flag","experian_id","panid")],function(x) c(min(x,na.rm=T),quantile(x,c(0.25),na.rm=T),mean(x,na.rm=T),median(x,na.rm=T), quantile(x,c(0.75),na.rm=T),max(x,na.rm=T),sum(x,na.rm=T)))
rownames(summary_num) <- c("min","Q1","mean","median","Q3","max","sum")

na_count <-sapply(finaldata_num[,!names(finaldata_num) %in% c("exposed_flag","experian_id","panid")], function(y) sum(length(which(is.na(y)))))
NAs  <- as.vector(na_count)
N <- nrow(finaldata)-na_count

#write csv
write.csv(t(rbind(N,NAs,summary_num)),paste0(output,"QCs/","Summary.csv"),na='')

# Create Crosstables for campaign and pre campaign period
sink(paste0(output,"QCs/","PreCampaign_Summary.txt")) 
CrossTable(finaldata$group,finaldata$Buyer_Pre_P1, expected = FALSE)
sink() 
sink(paste0(output,"QCs/","Campaign_Summary.txt")) 
CrossTable(finaldata$group,finaldata$Buyer_Pos_P1,expected = FALSE)
sink() 

# Create check list for factors and Buyers
buy.pre <-  unique(finaldata$Buyer_Pre_P1)
buy.pos <- unique(finaldata$Buyer_Pos_P1)
non.pre <- unique(finaldata$Nonbuyer_Pre_P1)
gr <- unique(finaldata$group)
mos <-  unique(finaldata$Mosaic)
ban <- unique(finaldata$banner)
exp.uniq <- length(unique(finaldata$experian_id))

# check freq for demographic variables
sink(paste0(output,"QCs/","Demographics_Summary.txt")) 
xtabs(~estimated_hh_income,finaldata)
cat("\n")
xtabs(~person_1_gender,finaldata)
cat("\n")
xtabs(~number_of_children_in_living_Un,finaldata)
cat("\n")
xtabs(~hh_age,finaldata)

cat("\n")
ftable(xtabs(~group+estimated_hh_income,finaldata))
cat("\n")
ftable(xtabs(~group+person_1_gender,finaldata))
cat("\n")
ftable(xtabs(~group+number_of_children_in_living_Un,finaldata))
cat("\n")
ftable(xtabs(~group+hh_age,finaldata))
sink()

finaldata_buyers <- finaldata[Buyer_Pos_P1=="1",]
sink(paste0(output,"QCs/","Demographics_Summary_Buyers_Campaign_Period.txt")) 
cat("\n")
xtabs(~estimated_hh_income,finaldata_buyers)
cat("\n")
xtabs(~person_1_gender,finaldata_buyers)
cat("\n")
xtabs(~number_of_children_in_living_Un,finaldata_buyers)
cat("\n")
xtabs(~hh_age,finaldata_buyers)

cat("\n")
ftable(xtabs(~group+estimated_hh_income,finaldata_buyers))
cat("\n")
ftable(xtabs(~group+person_1_gender,finaldata_buyers))
cat("\n")
ftable(xtabs(~group+number_of_children_in_living_Un,finaldata_buyers))
cat("\n")
ftable(xtabs(~group+hh_age,finaldata_buyers))
sink()

sink(paste0(output,"QCs/","Buyers Data Metrics.txt")) 
list(
  Mean_Dollars_Spent_per_Trip_POST_P1_Non_Exposed=mean(finaldata[group=="0" & Buyer_Pos_P1=="1",]$Dol_per_Trip_POS_P1),
  Mean_Dollars_Spent_per_Trip_PRE_P1_Non_Exposed=mean(finaldata[group=="0" & Buyer_Pos_P1=="1",]$Dol_per_Trip_PRE_P1),
  Mean_Dollars_Spent_per_Trip_POST_P1_Exposed=mean(finaldata[group!="0" & Buyer_Pos_P1=="1",]$Dol_per_Trip_POS_P1),
  Mean_Dollars_Spent_per_Trip_PRE_P1_Exposed=mean(finaldata[group!="0" & Buyer_Pos_P1=="1",]$Dol_per_Trip_PRE_P1),
  Mean_Trips_POST_P1_Non_Exposed=mean(finaldata[group=="0" & Buyer_Pos_P1=="1",]$Trps_POS_P1),
  Mean_Trips_PRE_P1_Non_Exposed=mean(finaldata[group=="0" & Buyer_Pos_P1=="1",]$Trps_PRE_P1),
  Mean_Trips_POST_P1_Exposed=mean(finaldata[group!="0" & Buyer_Pos_P1=="1",]$Trps_POS_P1),
  Mean_Trips_PRE_P1_Exposed=mean(finaldata[group!="0" & Buyer_Pos_P1=="1",]$Trps_PRE_P1),
  
  SD_Dol_per_Trip_POST_P1_Non_Exposed=sd(finaldata[group=="0" & Buyer_Pos_P1=="1",]$Dol_per_Trip_POS_P1),
  SD_Dol_per_Trip_PRE_P1_Non_Exposed=sd(finaldata[group=="0" & Buyer_Pos_P1=="1",]$Dol_per_Trip_PRE_P1),
  SD_Dol_per_Trip_POST_P1_Exposed=sd(finaldata[group!="0" & Buyer_Pos_P1=="1",]$Dol_per_Trip_POS_P1),
  SD_Dol_per_Trip_PRE_P1_Exposed=sd(finaldata[group!="0" & Buyer_Pos_P1=="1",]$Dol_per_Trip_PRE_P1),
  SD_Trips_POST_P1_Non_Exposed=sd(finaldata[group=="0" & Buyer_Pos_P1=="1",]$Trps_POS_P1),
  SD_Trips_PRE_P1_Non_Exposed=sd(finaldata[group=="0" & Buyer_Pos_P1=="1",]$Trps_PRE_P1),
  SD_Trips_POST_P1_Exposed=sd(finaldata[group!="0" & Buyer_Pos_P1=="1",]$Trps_POS_P1),
  SD_Trips_PRE_P1_Exposed=sd(finaldata[group!="0" & Buyer_Pos_P1=="1",]$Trps_PRE_P1),
  
  Min_Dol_per_Trip_POST_P1_Non_Exposed=min(finaldata[group=="0" & Buyer_Pos_P1=="1",]$Dol_per_Trip_POS_P1),
  Min_Dol_per_Trip_PRE_P1_Non_Exposed=min(finaldata[group=="0" & Buyer_Pos_P1=="1",]$Dol_per_Trip_PRE_P1),
  Min_Dol_per_Trip_POST_P1_Exposed=min(finaldata[group!="0" & Buyer_Pos_P1=="1",]$Dol_per_Trip_POS_P1),
  Min_Dol_per_Trip_PRE_P1_Exposed=min(finaldata[group!="0" & Buyer_Pos_P1=="1",]$Dol_per_Trip_PRE_P1),
  Min_Trips_POST_P1_Non_Exposed=min(finaldata[group=="0" & Buyer_Pos_P1=="1",]$Trps_POS_P1),
  Min_Trips_PRE_P1_Non_Exposed=min(finaldata[group=="0" & Buyer_Pos_P1=="1",]$Trps_PRE_P1),
  Min_Trips_POST_P1_Exposed=min(finaldata[group!="0" & Buyer_Pos_P1=="1",]$Trps_POS_P1),
  Min_Trips_PRE_P1_Exposed=min(finaldata[group!="0" & Buyer_Pos_P1=="1",]$Trps_PRE_P1),
  
  Max_Dol_per_Trip_POST_P1_Non_Exposed=max(finaldata[group=="0" & Buyer_Pos_P1=="1",]$Dol_per_Trip_POS_P1),
  Max_Dol_per_Trip_PRE_P1_Non_Exposed=max(finaldata[group=="0" & Buyer_Pos_P1=="1",]$Dol_per_Trip_PRE_P1),
  Max_Dol_per_Trip_POST_P1_Exposed=max(finaldata[group!="0" & Buyer_Pos_P1=="1",]$Dol_per_Trip_POS_P1),
  Max_Dol_per_Trip_PRE_P1_Exposed=max(finaldata[group!="0" & Buyer_Pos_P1=="1",]$Dol_per_Trip_PRE_P1),
  Max_Trips_POST_P1_Non_Exposed=max(finaldata[group=="0" & Buyer_Pos_P1=="1",]$Trps_POS_P1),
  Max_Trips_PRE_P1_Non_Exposed=max(finaldata[group=="0" & Buyer_Pos_P1=="1",]$Trps_PRE_P1),
  Max_Trips_POST_P1_Exposed=max(finaldata[group!="0" & Buyer_Pos_P1=="1",]$Trps_POS_P1),
  Max_Trips_PRE_P1_Exposed=max(finaldata[group!="0" & Buyer_Pos_P1=="1",]$Trps_PRE_P1)
)
sink()

cat("Please check summary.csv and proceed with the modeling process")


In [ ]:


#check if response and mandatory variables exist in the data
#Put also the sub campaigns as mandatory
#all_mandatory_vars <- c("Buyer_Pos_P1","Buyer_Pre_P1","Trps_POS_P1","Trps_PRE_P1","Dol_per_Trip_POS_P1","Dol_per_Trip_PRE_P1","Nonbuyer_Pre_P1","hh_age","estimated_hh_income","number_of_children_in_living_Un","person_1_gender","experian_id")
#print(all_mandatory_vars %in% names(finaldata))

# set exposed flag variable
#exposed_flag_var <- "exposed_flag"  
#XXX_Flag"
# set exposed flag variable
#exposed_flag_var <- "Exposed_Flag_2monthsplus"

# Put the sub campaigns and the demographic variables that will be used as random effects in the models
#random_demos <- c("estimated_hh_income","hh_age","number_of_children_in_living_Un")
#c("estimated_hh_income","hh_age","number_of_children_in_living_Un","person_1_gender")
#random_campaigns <- c("Creative_Groups")

# set extra exposed flag variable to be excluded (in case you have more than one in dataset) and publisher variables that are not used in the mixed models
#dropvars <- c("XXX")

# set True if P2 category products are competitor or False otherwise (need for sign check in the model)
#P2_Competitor <- TRUE #or FALSE

# pvalue threshold
#pvalue_lvl <- 0.20

# Create folders for outputs
#dir.create(paste0(output,"Descriptive Statistics for Publishers and Demographics"))
#dir.create(paste0(output,"Final_Modelling_Results"))
#dir.create(paste0(output,"Initial_Modelling_Results"))
#dir.create(paste0(output,"QCs"))
#dir.create(paste0(output,"saved_data"))
#dir.create(paste0(output,"Scoring"))
#final_modelling_output <- paste0(output,"Final_Modelling_Results/")
#initial_modelling_output <- paste0(output,"Initial_Modelling_Results/")
#scoring_output <- paste0(output,"Scoring/")
#dir.create(paste0(scoring_output,"Occasion/"))
#dir.create(paste0(scoring_output,"DollarsOccasion/"))
#dir.create(paste0(scoring_output,"Penetration/"))
#dir.create(paste0(final_modelling_output,"Occasion/"))
#dir.create(paste0(final_modelling_output,"DollarsOccasion/"))
#dir.create(paste0(final_modelling_output,"Penetration/"))
#dir.create(paste0(initial_modelling_output,"Occasion/"))
#dir.create(paste0(initial_modelling_output,"DollarsOccasion/"))

#scoring_Occ_output <- paste0(scoring_output,"Occasion/")
#scoring_DolOcc_output <- paste0(scoring_output,"DollarsOccasion/")
#scoring_Pen_output <- paste0(scoring_output,"Penetration/")
#final_Occ_modelling_output <- paste0(final_modelling_output,"Occasion/")
#final_DollOcc_modelling_output <- paste0(final_modelling_output,"DollarsOccasion/")
#final_Pen_modelling_output <- paste0(final_modelling_output,"Penetration/")
#initial_Occ_modelling_output <- paste0(initial_modelling_output,"Occasion/")
#initial_DollOcc_modelling_output <- paste0(initial_modelling_output,"DollarsOccasion/")


In [ ]:


#######################################
#--------- Data Manipulation ---------#
#######################################






#     for(i in random_campaigns)
#     {   
#       finaldata <- as.data.frame(finaldata)
#       finaldata[,i] <- as.factor(finaldata[,i])                                                  
#       
#       # re-group random intercept levels 
#       finaldata[,i] <- str_sub(finaldata[,i], start=1, end=2)
#       finaldata[,i] <- gsub("_","",finaldata[,i])
#       finaldata[,i] <- as.factor(finaldata[,i])
#       
#       # create a csv file with the counts of each level of the publisher variable
#       #write.csv(table(finaldata$group,finaldata[,eval(parse(text=random_campaigns[i]))]),file="ExposureFreq5w.csv")
#       
#       levelstokeep <- c("14","15","41","3","8","2","6","10","24")
#       all_levels <- unique(finaldata[,i])
#       
#       other_levels <- setdiff(all_levels,c(levelstokeep,"\\N"))
#       
#       finaldata <- as.data.table(finaldata)
#       for(j in other_levels){
#         finaldata <- finaldata[i==j,i:="Other"]
#       }
#       
#     }



# remove HHs with no gender info
finaldata <- finaldata[!person_1_gender=="U",]

# aggregate U and L levels of hh income
finaldata <- finaldata[estimated_hh_income=="U",estimated_hh_income:="L"]

panid_out_all <- c()
if(length(random_campaigns)>0){
  for(k in random_campaigns){
    finaldata <- finaldata[eval(parse(text=k))=="\\N"|eval(parse(text=k))=="NULL"|eval(parse(text=k))=="NONE",eval(parse(text=k)):="none"]
    cat(paste0(nrow(finaldata[(group=="0" & eval(parse(text=k))!="none")])," non exposed HHs with publisher info "))
    panid_out <- finaldata[(group=="0" & eval(parse(text=k))!="none"),panid]
    finaldata <- finaldata[!(group=="0" & eval(parse(text=k))!="none"),]  
    cat('\n')
    # check and remove HHs that were exposed and we had no publisher info
    cat(paste0(nrow(finaldata[(group!="0" & eval(parse(text=k))=="none")])," exposed HHs with no publisher info "))
    finaldata <- finaldata[!(group!="0" & eval(parse(text=k))=="none")]
    panid_out2 <- finaldata[(group!="0" & eval(parse(text=k))=="none"),panid]
    panid_out_all <- unique(c(panid_out_all,panid_out,panid_out2))
  }
}

# apply matching rules for control group vs exposed (default is TRUE)
Match_control_exposed_groups <- TRUE  #FALSE or TRUE

if(Match_control_exposed_groups==T & length(ProScore)==0){
  finaldata <- as.data.table(finaldata)
  data_exposed <- finaldata[group!="0" ,]
  data_nonexposed1 <- finaldata[group=="0" & Buyer_Pre_P1=="1",]
  data_nonexposed0 <- finaldata[group=="0" & Buyer_Pre_P1=="0",]
  
  if(nrow(finaldata[group=="0",])>2000000){
    new_data_dim <- 0.3*nrow(finaldata[group=="0",])  
  } else {
    if(nrow(finaldata[group=="0",])>1000000){
      new_data_dim <- 0.4*nrow(finaldata[group=="0",])
    } else {
      if(nrow(finaldata[group=="0",])>750000){
        new_data_dim <- 0.6*nrow(finaldata[group=="0",])
      } else {
        new_data_dim <- 0.7*nrow(finaldata[group=="0",])
      }
    }
  }
  
  dim_sample0 <- round(((new_data_dim-nrow(data_exposed))/(1+(nrow(data_exposed[Buyer_Pre_P1=="1",])/(nrow(data_exposed[Buyer_Pre_P1=="0",]))))))
  dim_sample1 <- new_data_dim-nrow(data_exposed)-dim_sample0
  
  new_data_nonexposed_11 <- as.data.frame(data_nonexposed1[sample(nrow(data_nonexposed1), dim_sample1, replace = TRUE),])
  new_data_nonexposed_01 <- as.data.frame(data_nonexposed0[sample(nrow(data_nonexposed0), dim_sample0,replace = TRUE),])
  data_exposed <- as.data.frame(data_exposed)
  finaldata_sample <- rbind(data_exposed,new_data_nonexposed_11,new_data_nonexposed_01)
  
  finaldata <- as.data.table(finaldata_sample)
}

if(Match_control_exposed_groups==T & length(ProScore)>0){
  exposed_proportion <- prop.table(xtabs(~eval(parse(text=ProScore)),finaldata[group!="0",]))
  proscore_levels <- names(exposed_proportion)
  
  if(nrow(finaldata[group=="0",])>2000000){
    new_data_dim <- 0.3*nrow(finaldata[group=="0",])  
  } else {
    if(nrow(finaldata[group=="0",])>1000000){
      new_data_dim <- 0.4*nrow(finaldata[group=="0",])
    } else {
      if(nrow(finaldata[group=="0",])>800000){
        new_data_dim <- 0.6*nrow(finaldata[group=="0",])
      } else {
        new_data_dim <- 0.7*nrow(finaldata[group=="0",])
      }
    }
  }
  
  sample_control_data <- c()
  
  for (i in proscore_levels){
    sample_i_dim <- round(new_data_dim*exposed_proportion[names(exposed_proportion)==i])
    temp_data <- finaldata[eval(parse(text=ProScore))==i & group=="0",]
    data_i <- temp_data[sample(nrow(temp_data), sample_i_dim, replace = TRUE),]
    sample_control_data <- rbind(sample_control_data,data_i)
  }
  
  sample_data <- rbind(sample_control_data,finaldata[group!="0",])
  
  sample_data <- as.data.table(sample_data)
  data_exposed <- as.data.table(sample_data[group!="0",])
  
  data_nonexposed1 <- sample_data[group=="0" & Buyer_Pre_P1=="1",]
  data_nonexposed0 <- sample_data[group=="0" & Buyer_Pre_P1=="0",]
  
  if(nrow(data_nonexposed1)/nrow(data_nonexposed0) > nrow(data_exposed[Buyer_Pre_P1=="1",])/nrow(data_exposed[Buyer_Pre_P1=="0",])){
    new_data_nonexposed_11 <- as.data.frame(data_nonexposed1[sample(nrow(data_nonexposed1), (nrow(data_exposed[Buyer_Pre_P1=="1",])/nrow(data_exposed[Buyer_Pre_P1=="0",]))*nrow(data_nonexposed0), replace = TRUE),])
    data_exposed <- as.data.frame(data_exposed)
    finaldata_sample <- rbind(data_exposed,new_data_nonexposed_11,data_nonexposed0)
  } else {
    new_data_nonexposed_01 <- as.data.frame(data_nonexposed0[sample(nrow(data_nonexposed0), (nrow(data_exposed[Buyer_Pre_P1=="0",])/nrow(data_exposed[Buyer_Pre_P1=="1",]))*nrow(data_nonexposed1),replace = TRUE),])
    data_exposed <- as.data.frame(data_exposed)
    finaldata_sample <- rbind(data_exposed,new_data_nonexposed_01,data_nonexposed1)
  }
  
  finaldata <- as.data.table(finaldata_sample)
}

# check proportions of the sample dataset
if(Match_control_exposed_groups==T ){
  finaldata_sample <- as.data.table(finaldata_sample)
  sink(paste0(output,"Matching_data_summary.txt"))
  if(length(ProScore)>0){
    cat("\n")
    xt <- xtabs(~eval(parse(text=ProScore)),finaldata_sample[group!="0",])
    names(dimnames(xt)) <- c("Proportions of ProScore in the exposed group")
    print(prop.table(xt))
    cat("\n\n")
    xt <- xtabs(~eval(parse(text=ProScore)),finaldata_sample[group=="0",])
    names(dimnames(xt)) <- c("Proportions of ProScore in the control group")
    print(prop.table(xt))
  }
  cat("\n\n")
  xt <- xtabs(~Buyer_Pre_P1,finaldata_sample[group!="0",])
  names(dimnames(xt)) <- c("Percentages of target product buyers during the Pre-Campaign Period in the exposed group")
  print(prop.table(xt))
  cat("\n\n")
  xt <- xtabs(~Buyer_Pre_P1,finaldata_sample[group=="0",])
  names(dimnames(xt)) <- c("Percentages of target product buyers during the Pre-Campaign Period in the control group")
  print(prop.table(xt))
  cat("\n\n")
  cat('Average_Trips_PRE_P1_Non_Exposed_matched_data')
  cat("\n")
  cat(mean(finaldata_sample[Buyer_Pre_P1=="1" & group=="0",]$Trps_PRE_P1))
  cat("\n\n")
  cat('Average_Trips_PRE_P1__Exposed_matched_data')
  cat("\n")
  cat(mean(finaldata_sample[Buyer_Pre_P1=="1" & group!="0",]$Trps_PRE_P1))
  cat("\n\n")
  cat('Average_Spent_PRE_P1_Non_Exposed_matched_data')
  cat("\n")
  cat(mean(finaldata_sample[Buyer_Pre_P1=="1" & group=="0",]$Dol_per_Trip_PRE_P1))
  cat("\n\n")
  cat('Average_Spent_PRE_P1_Exposed_matched_data')
  cat("\n")
  cat(mean(finaldata_sample[Buyer_Pre_P1=="1" & group!="0",]$Dol_per_Trip_PRE_P1))
  sink()
}

# remove demographic variables
finaldata <- droplevels(finaldata)
finaldata$panid <- as.character(finaldata$panid)
names(finaldata)
initial_data <- finaldata[,c(4:(which(names(finaldata)=="state")-1),which(names(finaldata)=="person_1_gender"),which(names(finaldata)=="number_of_children_in_living_Un"),(which(names(finaldata)=="Mosaic")+1):length(names(finaldata))),with=F]
names(initial_data)
rm(finaldata,data_nonexposed1,temp_data,data_i,sample_control_data,sample_data,data_exposed,data_nonexposed0,finaldata_sample,finaldata_num,new_data_nonexposed_11,new_data_nonexposed_01)
# get number of products in the data
num_products <- length(grep("Buyer_Pre_P",colnames(initial_data)))-1

# mandatory variables for scoring
scoring_vars <- c("Prd_1_Net_Pr_PRE","Prd_1_Net_Pr_POS","Buyer_Pos_P0","Buyer_Pre_P0")

#exclude POST variables
indexPOS <- grep("POS",colnames(initial_data))
indexPos <- grep("Pos",colnames(initial_data))
indexPre1 <- grep("Buyer_Pre_",colnames(initial_data))
indexPre2 <- grep("Nonbuyer_Pre_",colnames(initial_data))
# check pre vars to exclude (only buyer_pre and nonbuyer_pre should be excluded)
(colnames(initial_data)[c(indexPre1,indexPre2)])
initial_vars_to_exclude <- colnames(initial_data)[c(indexPOS,indexPos,indexPre1,indexPre2)]
indexPREPOS <- grep("PRE_POS",colnames(initial_data))
indexPrePos <- grep("Pre_Pos",colnames(initial_data))
PREPOS_vars_to_include <- colnames(initial_data)[c(indexPREPOS,indexPrePos)]
ALL_vars_to_exclude <- setdiff(initial_vars_to_exclude,c(PREPOS_vars_to_include,all_mandatory_vars,scoring_vars))
initial_data <- as.data.frame(initial_data)
initial_data <- initial_data[ , !names(initial_data) %in% c(ALL_vars_to_exclude)] 

#exclude category variables(exclude P0's)
indexP0 <- grep("0",colnames(initial_data))
P0_vars_to_exclude <- colnames(initial_data)[indexP0]
indexMODEL <- grep("MODEL",colnames(initial_data))
ProScore_vars_to_include <- colnames(initial_data)[indexMODEL]
P0_vars_to_exclude <- setdiff(P0_vars_to_exclude,c(ProScore_vars_to_include,all_mandatory_vars,scoring_vars))
initial_data <- initial_data[ , !names(initial_data) %in% c(P0_vars_to_exclude)] 

#exclude new variables that are only for reporting
indexPERC <- grep("Perc_",colnames(initial_data))
indexPriPerUn <- grep("Pr_per_",colnames(initial_data))
vars_to_exclude <- colnames(initial_data)[c(indexPERC,indexPriPerUn)]
initial_data <- initial_data[ , !names(initial_data) %in% c(vars_to_exclude)] 

# segments for outliers detection
initial_data <- as.data.table(initial_data)
data_NB_NE_B <- initial_data[Buyer_Pre_P1=="0" & group=="0"  & Buyer_Pos_P1=="1",]
data_B_E_NB <- initial_data[Buyer_Pre_P1=="1" & group!="0" & Buyer_Pos_P1=="0",]
pen_reduction <- initial_data[Buyer_Pre_P1=="1" & group=="0" & Buyer_Pos_P1=="0",]
occ_reduction <- initial_data[group=="0" & Buyer_Pos_P1=="1" & Trps_POS_P1<Trps_PRE_P1,]
dol_reduction <- initial_data[group=="0" & Buyer_Pos_P1=="1" & Dol_per_Trip_POS_P1<Dol_per_Trip_PRE_P1,]

# get variables that need to have negative sign
negative_index <- c()
for(i in 3:num_products){
  indexCompetitorVars <- grep(paste0("P",i),colnames(initial_data))  
  indexCompetitorVars2 <- grep(paste0(i,"_"),colnames(initial_data)) 
  indexGAS <- grep("AVG_PRICE",colnames(initial_data)) 
  negative_index <- c(negative_index,indexCompetitorVars,indexCompetitorVars2,indexGAS)
}
negativevars <- unique(colnames(initial_data)[negative_index])

if(P2_Competitor==T){
  indexNOCompetitorVars1 <- grep("P2",colnames(initial_data))
  indexNOCompetitorVars2 <- grep(paste0(2,"_"),colnames(initial_data)) 
  indexNOCompetitorVars <- c(indexNOCompetitorVars1,indexNOCompetitorVars2)
  NOCompetitorVars <- colnames(initial_data)[indexNOCompetitorVars]
  negativevars <- unique(c(negativevars,NOCompetitorVars))
}

# get variables that need to have positive sign
positive_index1 <- grep("P1",colnames(initial_data))  
positive_index2 <- grep("1_",colnames(initial_data))  
positive_index3 <- grep("MODEL",colnames(initial_data)) 
positive_index <- c(positive_index1,positive_index2,positive_index3)
positivevars <- colnames(initial_data)[positive_index]

# # kruskal wallis test for pair comparison of publisher's levels keep aggregated levels based on the spent aggregation
# kruskal_var <- 'Dol_per_Trip_POS_P1'
# buyers_data <- initial_data[Buyer_Pos_P1=='1',]
# kruskal_randoms <- c()
# for(i in random_campaigns){
#   kt <- kruskal.test(eval(parse(text=kruskal_var))~eval(parse(text=i)),data=buyers_data)
#   cat(i);print(kt)
#   if(kt$p.value<0.10){
#     kruskal_randoms <- append(kruskal_randoms,i)
#   } else {
#     initial_data[,eval(parse(text=i)):=NULL]
#   }
# }
# 
# warning('In case of aggregation based on Kruskal-Wallis test, make sure that the other level of each sub-campaign is named as Other_XXX')
# for(i in kruskal_randoms){
#   lev <- setdiff(levels(buyers_data[,eval(parse(text=i))]),'none')
#   if(length(lev)>2){
#     aggr <- c()
#     for(j in 1:(length(lev)-1)){
#       for( k in (j+1):(length(lev))){       
#         df <- subset(buyers_data, eval(parse(text=i))  %in% c(lev[j],lev[k]))
#         l1 <- list(c(lev[j],lev[k]),P_Value=try(round(kruskal.test(df[,eval(parse(text=kruskal_var))]~df[,eval(parse(text=i))])$p.value,5),silent=TRUE))
#         if(l1[2]=="NaN") next
#         vectorElements <- unlist(l1)
#         listnames <- names(vectorElements) <- c(paste0(i,':'),paste0(i,':'),'P_value:')
#         listout <- paste(listnames,vectorElements)
#         write.table(listout,file=paste0(output,'Kruskal.csv'), row.names = FALSE, col.names = FALSE, quote = FALSE,append=TRUE)  
#         if(l1[['P_Value']]<0.05){
#           aggr <- append(aggr,c(sapply(l1[1],`[`,1),sapply(l1[1],`[`,2)))
#           aggr <- setdiff(aggr,c('none',paste0("Other_",i)))
#         }
#       }  
#     }
#     if(length(unique(aggr))>0){
#       initial_data[!eval(parse(text=i)) %in% c(aggr,'none'),eval(parse(text=i)):=paste0("Other_",i)]
#     } else {
#       initial_data[,eval(parse(text=i)):=NULL]
#     }
#   }
# }

initial_data <- droplevels(initial_data)
saveRDS(initial_data,"initial_data.RDS")
random_campaigns <- intersect(random_campaigns,names(initial_data))

if(length(random_campaigns)>0){
  random_campaigns_factors <- paste0("+(0+group|",random_campaigns,")")
} else {
  random_campaigns_factors <- NULL
}
random_demos_factors <- paste0( "+(0+group|",random_demos,")" )

# Create Descriptives for Campaign breaks
if(length(random_campaigns)>0){
  output_checks <- paste0(output,"Descriptive Statistics for Publishers and Demographics/")
  randoms <- c(random_demos,random_campaigns)
  for(i in randoms){
    write.csv(table(initial_data[,eval(parse(text=i))]),file=paste0(output_checks,i,'.csv'),row.names=F)
    write.csv(table(initial_data$group,initial_data[,eval(parse(text=i))]),file=paste0(output_checks,i,'_group.csv'))
    
    dolocc_data <- initial_data[Buyer_Pos_P1=="1",]
    
    write.csv(table(dolocc_data[,eval(parse(text=i))]),file=paste0(output_checks,i,'_buyers.csv'),row.names=F)
    write.csv(table(dolocc_data$group,dolocc_data[,eval(parse(text=i))]),file=paste0(output_checks,i,'_group_buyers.csv')) 
  }
  
  # check exposed and non exposed HHs counts for target brand Buyers in the campaign period
  write.csv(table(initial_data$group,initial_data$Buyer_Pos_P1),file=paste0(output_checks,'buyers_pos_group.csv'))
  
  # check exposed and non exposed HHs counts for target brand Buyers in the pre-campaign period
  write.csv(table(initial_data$group,initial_data$Buyer_Pre_P1),file=paste0(output_checks,'buyers_pre_group.csv'))
  
  # calculate correlations between levels of the random intercept variables
  finaldata_cor <- as.data.frame(initial_data)
  
  for(i in randoms){
    for(level in unique(eval(parse(text=paste0("finaldata_cor$",i))))){
      finaldata_cor[paste(i, level, sep = "_")] <- ifelse(eval(parse(text=paste0("finaldata_cor$",i))) == level, 1, 0)
    }
  }
  
  cor.data <- finaldata_cor[,c((which(names(finaldata_cor)=="panid")+1):length(finaldata_cor))]
  cor.matrix <- cor(cor.data)
  write.csv(cor.matrix,file=paste0(output_checks,"random_correlation_matrix.csv"))
  
  mel.cc <- melt(cor.matrix)
  heat_map <- ggplot(mel.cc,aes(Var1, Var2))+geom_tile(aes(fill=value))+ scale_fill_viridis(name="Correlation",option="magma") + theme(axis.text.x = element_text(angle=90, vjust=0.5))+ coord_equal() 
  ggsave(filename=paste0(output_checks,"correlation_heat_map.pdf"),heat_map,width = 10, height = 10, units = "in")
  
  #Make descriptive plots for publishers, creatives etc.
  # create descriptive plots for the DolOcc breaks
  buyers_data <- initial_data[Buyer_Pos_P1=="1",]
  for(i in random_campaigns){
    buyers_data <- buyers_data[eval(parse(text=i))!="none",]
    buyers_data <- droplevels(buyers_data)
    buyers_data <- as.data.frame(buyers_data)
    res <- ldply(levels(buyers_data[,i]),function(x) round(mean(buyers_data[buyers_data[,i]==x,]$Dol_per_Trip_POS_P1),2))
    df <-cbind(res,levels(buyers_data[,i]))
    colnames(df) <- c('Average_Spent_during_the_campaign_period','Levels')
    
    p <- ggplot(df,aes(x=Levels,y=Average_Spent_during_the_campaign_period))+geom_bar(stat='identity')+ theme(axis.text.x = element_text(angle=90, vjust=0.5))+ylab('Average Spent for the target brand during the campaign period')+
      theme(legend.position="none")+theme(panel.background = element_rect(fill = 'white'))+
      geom_text(stat='identity',aes(label=Average_Spent_during_the_campaign_period,group=Levels), position = position_dodge(0.9), vjust = 0)  
    ggsave(filename=paste0(output_checks,i,'_Mean_Spent.pdf'),p)
    
    p <- ggplot(buyers_data,aes_string(i,'Dol_per_Trip_POS_P1'))+geom_boxplot()+theme(axis.text.x = element_text(angle = 90, vjust = 0.5))+
      ylab('Spent per Trip for the target bramd during the campaign period')
    ggsave(filename=paste0(output_checks,i,'_Spent_BoxPlot.pdf'),p)
    buyers_data <- as.data.table(buyers_data)
  } 
  
  # create descriptive plots for the Occ breaks
  options(warn=-1)
  a1 <- buyers_data[Trps_POS_P1>=5,Trps_POS_P1:="5"]
  options(warn=0)
  a1[,Trips_growth:=log(Trps_POS_P1/(Trps_PRE_P1+1))]
  a1$Trps_POS_P1 <- as.factor(a1$Trps_POS_P1)
  a1 <- a1[Trps_POS_P1=="5",Trps_POS_P1:="5+"]
  
  response_var <- c("Trps_POS_P1")
  for(i in random_campaigns){
    a1 <- as.data.table(a1)
    a1 <- a1[eval(parse(text=i))!="none",]
    a1 <- droplevels(a1)
    rr <- c(response_var,i)
    a1 <- as.data.frame(a1)
    df <- a1[,rr]
    mel <- melt(df,id.var=i)
    p <- ggplot(mel,aes_string(i))+geom_bar(aes(y=..count..,fill=as.factor(value)),position=position_dodge(0.9))+
      theme(axis.text.x = element_text(angle = 90, vjust = 0.5))+guides(fill=guide_legend(title=response_var))+
      geom_text(stat='count',aes(label=..count..,group=value), position = position_dodge(0.9), vjust = 0)  
    ggsave(filename=paste0(output_checks,i,'_Trips.pdf'),p)
    
    a2 <- as.data.table(initial_data[Buyer_Pos_P1=="1",])
    a2 <- a2[eval(parse(text=i))!="none",]
    a2 <- as.data.frame(a2)
    a2 <- droplevels(a2)
    res <- ldply(levels(a2[,i]),function(x) round(mean(a2[a2[,i]==x,]$Trps_POS_P1),2))
    df <-cbind(res,levels(a2[,i]))
    colnames(df) <- c('Average_Trips_during_the_campaign_period','Levels')
    p <- ggplot(df,aes(x=Levels,y=Average_Trips_during_the_campaign_period))+geom_bar(stat='identity')+ theme(axis.text.x = element_text(angle=90, vjust=0.5))+ylab('Average Trips for the target brand during the campaign period')+
      theme(legend.position="none")+theme(panel.background = element_rect(fill = 'white'))+
      geom_text(stat='identity',aes(label=Average_Trips_during_the_campaign_period,group=Levels), position = position_dodge(0.9), vjust = 0)  
    ggsave(filename=paste0(output_checks,i,'_Mean_Trips.pdf'),p)
  }
  
  # create descriptive plots for the Pen breaks
  response_var <- c("Buyer_Pos_P1")
  for(i in random_campaigns){
    data <- as.data.table(copy(initial_data))
    data <- data[eval(parse(text=i))!="none",]
    data <- droplevels(data)
    data <- as.data.frame(data)
    rr <- c(response_var,i)
    df <- data[,rr]
    mel <- melt(df,id.var=i)
    p <- ggplot(mel,aes_string(x=i))+geom_bar(aes(y=..count..,fill=as.factor(value)),position=position_dodge(0.9))+
      theme(axis.text.x = element_text(angle = 90, vjust = 0.5))+guides(fill=guide_legend(title=response_var))+
      geom_text(stat='count',aes(label=..count..,group=value), position = position_dodge(0.9), vjust = 0)  
    ggsave(paste0(output_checks,i,'_Buyers_during_Campaign.pdf'),p)  
  }
}

rm(dolocc_data,mel.cc,finaldata_cor,cor.data,cor.matrix,buyers_data,a1,a2,data)

#Create functions for modelling process and outputs
ggCaterpillar <- function(re,  QQ=FALSE, likeDotplot=FALSE) {
  require(ggplot2)
  f <- function(x) {
    pv   <- attr(x, "postVar")
    cols <- 1:(dim(pv)[1])
    se   <- unlist(lapply(cols, function(i) sqrt(pv[i, i, ])))
    ord  <- unlist(lapply(x, order)) + rep((0:(ncol(x) - 1)) * nrow(x), each=nrow(x))
    pDf  <- data.frame(y=unlist(x)[ord],
                       ci=1.28*se[ord], #80%CI:1.28, 90%CI:1.645, 95%CI: 1.96
                       nQQ=rep(qnorm(ppoints(nrow(x))), ncol(x)),
                       ID=factor(rep(rownames(x), ncol(x))[ord], levels=rownames(x)[ord]),
                       ind=gl(ncol(x), nrow(x), labels=names(x)))
    
    if(QQ) {  ## normal QQ-plot
      p <- ggplot(pDf, aes(nQQ, y))
      p <- p + facet_wrap(~ ind, scales="free")
      p <- p + xlab("Standard normal quantiles") + ylab("Random effect quantiles")
    } else {  ## caterpillar dotplot
      p <- ggplot(pDf, aes(ID, y)) #+ coord_flip()
      if(likeDotplot) {  ## imitate dotplot() -> same scales for random effects
        p <- p + facet_wrap(~ ind)
      } else {           ## different scales for random effects
        p <- p + facet_wrap(~ ind, scales="free_y")
      }
      p <- p + xlab("Levels") + ylab("Random effects")
    }
    
    p <- p + theme(legend.position="none")
    p <- p + geom_hline(yintercept=0)
    p <- p + geom_errorbar(aes(ymin=y-ci, ymax=y+ci), width=0, colour="black")
    p <- p + geom_point(aes(size=1.2), colour="blue") + theme(axis.text.x = element_text(angle=90, vjust=0.5)) 
    
    return(p)
    #ggsave(paste0(output,"dotplots.pdf"),p)
  }
  
  lapply(re,f)
}

se.ranef <- function (object) {   
  se.bygroup <- ranef(object, condVar = TRUE) 
  n.groupings <- length(se.bygroup) 
  for (m in 1:n.groupings) {        
    vars.m <- attr(se.bygroup[[m]], "postVar")  
    K <- dim(vars.m)[1]   
    J <- dim(vars.m)[3] 
    se.bygroup[[m]] <- array(NA, c(J, K))   
    for (j in 1:J) {          
      se.bygroup[[m]][j, ] <- sqrt(diag(as.matrix(vars.m[,, j])))  }        
    names.full <- dimnames(se.bygroup)      
    dimnames(se.bygroup[[m]]) <- list(names.full[[1]], names.full[[2]]) }
  return(se.bygroup) }

# create nloptwrap optimization method
defaultControl <- list(algorithm="NLOPT_LN_BOBYQA",xtol_rel=1e-6,maxeval=1e5)
nloptwrap2 <- function(fn,par,lower,upper,control=list(),...) {
  for (n in names(defaultControl)) 
    if (is.null(control[[n]])) control[[n]] <- defaultControl[[n]]
    res <- nloptr(x0=par,eval_f=fn,lb=lower,ub=upper,opts=control,...)
    with(res,list(par=solution,
                  fval=objective,
                  feval=iterations,
                  conv=if (status>0) 0 else status,
                  message=message))
}

augment.ranef.mer <- function(x,
                              ci.level=0.9,
                              reorder=TRUE,
                              order.var=1) {
  tmpf <- function(z) {
    if (is.character(order.var) && !order.var %in% names(z)) {
      order.var <- 1
      warning("order.var not found, resetting to 1")
    }
    ## would use plyr::name_rows, but want levels first
    zz <- data.frame(level=rownames(z),z,check.names=FALSE)
    if (reorder) {
      ## if numeric order var, add 1 to account for level column
      ov <- if (is.numeric(order.var)) order.var+1 else order.var
      zz$level <- reorder(zz$level, zz[,order.var+1], FUN=identity)
    }
    ## Q-Q values, for each column separately
    qq <- c(apply(z,2,function(y) {
      qnorm(ppoints(nrow(z)))[order(order(y))]
    }))
    rownames(zz) <- NULL
    pv   <- attr(z, "postVar")
    cols <- 1:(dim(pv)[1])
    se   <- unlist(lapply(cols, function(i) sqrt(pv[i, i, ])))
    ## n.b.: depends on explicit column-major ordering of se/melt
    zzz <- cbind(melt(zz,id.vars="level",value.name="estimate"),
                 qq=qq,std.error=se)
    ## reorder columns:
    subset(zzz,select=c(variable, level, estimate, qq, std.error))
  }
  dd <- ldply(x,tmpf,.id="grp")
  ci.val <- -qnorm((1-ci.level)/2)
  transform(dd,
            p=2*pnorm(-abs(estimate/std.error)), ## 2-tailed p-val
            lb=estimate-ci.val*std.error,
            ub=estimate+ci.val*std.error)
}

#####################################
#--------------DOL_OCC--------------#
#####################################

dolocc.start.time <- Sys.time()

#set response variable
y_var <- "Dol_per_Trip_POS_P1"

# keep only the buyers
finaldata_dolocc <- initial_data[Buyer_Pos_P1=="1",]

# drop variables with only one level
indices <- sapply(finaldata_dolocc,function(x) length(unique(x))==1)
finaldata_dolocc <- finaldata_dolocc[,!indices,with=F]

#######################################
#-------- Variable Selection ---------#
#######################################

# #kruskal wallis test for pair comparison of publisher's levels for DolOcc model only
# #DolOcc model
# # kruskal wallis test for pair comparison of publisher's levels
# kruskal_var <- 'Dol_per_Trip_POS_P1'
# kruskal_randoms <- c()
# for(i in random_campaigns){
#   kt <- kruskal.test(eval(parse(text=kruskal_var))~eval(parse(text=i)),data=finaldata_dolocc)
#   cat(i);print(kt)
#   if(kt$p.value<0.10){
#     kruskal_randoms <- append(kruskal_randoms,i)
#   } else {
#     finaldata_dolocc[,eval(parse(text=i)):=NULL]
#   }
# }
#
# warning('In case of aggregation based on Kruskal-Wallis test, make sure that the other level of each sub-campaign is named as Other_XXX')
# for(i in kruskal_randoms){
#   lev <- setdiff(levels(finaldata_dolocc[,eval(parse(text=i))]),'none')
#   if(length(lev)>2){
#     aggr <- c()
#     for(j in 1:(length(lev)-1)){
#       for( k in (j+1):(length(lev))){       
#         df <- subset(finaldata_dolocc, eval(parse(text=i))  %in% c(lev[j],lev[k]))
#         l1 <- list(c(lev[j],lev[k]),P_Value=try(round(kruskal.test(df[,eval(parse(text=kruskal_var))]~df[,eval(parse(text=i))])$p.value,5),silent=TRUE))
#         if(l1[2]=="NaN") next
#         vectorElements <- unlist(l1)
#         listnames <- names(vectorElements) <- c(paste0(i,':'),paste0(i,':'),'P_value:')
#         listout <- paste(listnames,vectorElements)
#         write.table(listout,file=paste0(output,'Kruskal.csv'), row.names = FALSE, col.names = FALSE, quote = FALSE,append=TRUE)  
#         if(l1[['P_Value']]<0.05){
#           aggr <- append(aggr,c(sapply(l1[1],`[`,1),sapply(l1[1],`[`,2)))
#           aggr <- setdiff(aggr,c('none',paste0("Other_",i)))
#         }
#       }  
#     }
#     if(length(unique(aggr))>0){
#       finaldata_dolocc[!eval(parse(text=i)) %in% c(aggr,'none'),eval(parse(text=i)):=paste0("Other_",i)]
#     } else {
#       finaldata_dolocc[,eval(parse(text=i)):=NULL]
#     }
#   }
# }

# finaldata_dolocc <- droplevels(finaldata_dolocc)
# dolocc_random_campaigns <- intersect(random_campaigns,names(finaldata_dolocc))
# 
# if(length(dolocc_random_campaigns)>0){
#   dolocc_random_campaigns_factors <- paste0("+(0+group|",dolocc_random_campaigns,")")
# } else {
#   dolocc_random_campaigns_factors <- NULL
# }
# random_demos_factors <- paste0( "+(0+group|",random_demos,")" )

# Create Descriptives for Campaign breaks in case of running Kruskal in DolOcc above
#dir.create(paste0(initial_DollOcc_modelling_output,"Descriptives/"))
#output_checks <- paste0(initial_DollOcc_modelling_output,"Descriptives/")
#randoms <- c(dolocc_random_campaigns)
# # create descriptive plots for the DolOcc breaks
# for(i in dolocc_random_campaigns){
#   finaldata_dolocc <- finaldata_dolocc[eval(parse(text=i))!="none",]
#   finaldata_dolocc <- droplevels(finaldata_dolocc)
#   finaldata_dolocc <- as.data.frame(finaldata_dolocc)
#   res <- ldply(levels(finaldata_dolocc[,i]),function(x) round(mean(finaldata_dolocc[finaldata_dolocc[,i]==x,]$Dol_per_Trip_POS_P1),2))
#   df <-cbind(res,levels(finaldata_dolocc[,i]))
#   colnames(df) <- c('Average_Spent_during_the_campaign_period','Levels')
#   
#   p <- ggplot(df,aes(x=Levels,y=Average_Spent_during_the_campaign_period))+geom_bar(stat='identity')+ theme(axis.text.x = element_text(angle=90, vjust=0.5))+ylab('Average Spent for the target brand during the campaign period')+
#     theme(legend.position="none")+theme(panel.background = element_rect(fill = 'white'))+
#     geom_text(stat='identity',aes(label=Average_Spent_during_the_campaign_period,group=Levels), position = position_dodge(0.9), vjust = 0)  
#   ggsave(filename=paste0(output_checks,i,'_Mean_Spent.pdf'),p)
#   
#   p <- ggplot(finaldata_dolocc,aes_string(i,'Dol_per_Trip_POS_P1'))+geom_boxplot()+theme(axis.text.x = element_text(angle = 90, vjust = 0.5))+
#     ylab('Spent per Trip for the target bramd during the campaign period')
#   ggsave(filename=paste0(output_checks,i,'_Spent_BoxPlot.pdf'),p)
# } 

dolocc_mandatory_covariate <- c("Nonbuyer_Pre_P1")

# set initial model formula
finaldata_dolocc <- as.data.frame(finaldata_dolocc)
finaldata_dolocc <- finaldata_dolocc[, !(names(finaldata_dolocc) %in% c("Trps_POS_P1","Buyer_Pre_P1","Trps_PRE_P1"))]
Formula_fixed <- paste0(paste0(y_var,"~ ",paste0(names(finaldata_dolocc[,!names(finaldata_dolocc)%in%c(y_var,"panid","Dol_per_Trip_PRE_P1",random_demos,random_campaigns,scoring_vars)]),collapse=" + ")," ")," + offset(log(Dol_per_Trip_PRE_P1+1))")
options(warn=-1)

# run glm model
model_glm_total <- glm(Formula_fixed,data=finaldata_dolocc,family=Gamma(link="log"))
summary(model_glm_total)

##### Variable selection - Condition 1 #####
# Remove pvalues greater than 0.7 and rerun the glm model
pval_excluded_vars <- setdiff(c(rownames(as.data.frame(which(summary(model_glm_total)$coefficient[,4]>0.7))),setdiff(names(model_glm_total$coefficients),rownames(summary(model_glm_total)$coefficients))),dolocc_mandatory_covariate)
finaldata_dolocc <- finaldata_dolocc[, !names(finaldata_dolocc) %in% c(pval_excluded_vars)] 

# re-run glm model
Formula_fixed <- paste0(paste0(y_var,"~ ",paste0(names(finaldata_dolocc[,!names(finaldata_dolocc)%in%c(y_var,"panid","Dol_per_Trip_PRE_P1",random_demos,random_campaigns,scoring_vars)]),collapse=" + ")," ")," + offset(log(Dol_per_Trip_PRE_P1+1))")
model_glm_condition1 <- glm(Formula_fixed,data=finaldata_dolocc,family=Gamma(link="log"))
summary(model_glm_condition1)
rm(model_glm_total)

##### Variable selection - Condition 2 #####
# Remove variables that have z value lower than 2 and VIF greater than 15

# get variables with z value lower than 2
z_excluded_vars  <- rownames(as.data.frame(which(abs(summary(model_glm_condition1)$coefficient[,3])<1.96)))

# get the variables with VIF greater than 15
multicol <- alias(model_glm_condition1)
multicols_toremove <- rownames(as.data.frame(multicol$Complete))

if (length(multicols_toremove)>0){
  finaldata_dolocc <- finaldata_dolocc[ , !names(finaldata_dolocc) %in% multicols_toremove]
  Formula_fixed <- paste0(paste0(y_var,"~ ",paste0(names(finaldata_dolocc[,!names(finaldata_dolocc)%in%c(y_var,"panid")]),collapse=" + ")," ")," + offset(log(Dol_per_Trip_PRE_P1+1))")
  model_glm_new <- glm(Formula_fixed,data=finaldata_dolocc,family=Gamma(link="log"))
} else {
  model_glm_new <- model_glm_condition1
}

vif_output <- (vif(model_glm_new))
vif_to_remove <- names(vif_output[vif_output > 15])

# final vars to exclude under condition 2
vars_condition2 <- setdiff(intersect(z_excluded_vars,vif_to_remove),dolocc_mandatory_covariate)
finaldata_dolocc <- finaldata_dolocc[ , !names(finaldata_dolocc) %in% vars_condition2] 

# re-run glm model
Formula_fixed <- paste0(paste0(y_var,"~ ",paste0(names(finaldata_dolocc[,!names(finaldata_dolocc)%in%c(y_var,"panid","Dol_per_Trip_PRE_P1",random_demos,random_campaigns,scoring_vars)]),collapse=" + ")," ")," + offset(log(Dol_per_Trip_PRE_P1+1))")
model_glm <- glm(Formula_fixed,data=finaldata_dolocc,family=Gamma(link="log"))
summary(model_glm)
rm(model_glm_condition1)

# p-values and signs checks
neutralvars <- setdiff(names(finaldata_dolocc),c(positivevars,negativevars))
pvalues <- summary(model_glm)$coefficients[,4][-1]
keep_covariate <- names(pvalues[pvalues<pvalue_lvl])
coeffs <- sign(summary(model_glm)$coefficients[,1])
varstokeep <- c();initialvars <- c()
for (i in negativevars) if (i %in% names(coeffs[coeffs<0])) varstokeep <- c(varstokeep,i)
for (j in positivevars) if (j %in% names(coeffs[coeffs>0])) varstokeep <- c(varstokeep,j)
for (l in neutralvars) varstokeep <- c(varstokeep,l)
for (k in varstokeep) if (k %in% keep_covariate) initialvars <- c(initialvars,k)
if(dolocc_mandatory_covariate %in% names(finaldata_dolocc) & length(setdiff(dolocc_mandatory_covariate,initialvars))>0){
  initialvars <- c(dolocc_mandatory_covariate,initialvars)
}

#set the formula for the second model after cleaning the first model
Formula_fixed <- paste0(paste0(y_var,"~ ",paste0(initialvars,collapse=" + ")," ")," + group + offset(log(Dol_per_Trip_PRE_P1+1))")
model_glm_2 <-  glm(Formula_fixed,data=finaldata_dolocc,family=Gamma(link="log"))
summary(model_glm_2)
rm(model_glm)

# final cleaning of second model based on signs and pvalues
pvalues_2 <- summary(model_glm_2)$coefficients[,4][-1]
keep_covariate_2 <- names(pvalues_2[pvalues_2 < pvalue_lvl])
coeffs_2 <- sign(summary(model_glm_2)$coefficients[,1])
varstokeep <- c();vars_2 <- c()
for (i in negativevars) if (i %in% names(coeffs_2[coeffs_2 < 0])) varstokeep <- c(varstokeep,i)
for (j in positivevars) if (j %in% names(coeffs_2[coeffs_2 > 0])) varstokeep <- c(varstokeep,j)
for (l in neutralvars) varstokeep <- c(varstokeep,l)
for (k in varstokeep) if (k %in% keep_covariate_2) vars_2 <- c(vars_2,k)

# final correlation check (find pair vars with correlation greater then 0.8 and drop the one with bigger pvalue)
if(length(vars_2)>1){
  finaldata_dolocc_num <- as.data.table(finaldata_dolocc[,names(finaldata_dolocc) %in% vars_2])
  t1 <- as.data.frame(finaldata_dolocc_num[,sapply(finaldata_dolocc_num,is.numeric)])
  finaldata_dolocc_num <- as.data.frame(finaldata_dolocc_num)
  finaldata_dolocc_num <- finaldata_dolocc_num[,c(which(t1==T))]
  temp_cor <- as.matrix(cor(finaldata_dolocc_num))
  temp_cor[lower.tri(temp_cor, diag = T)] <- NA
  temp_corIndex <- which(temp_cor > 0.8 |temp_cor < (-0.8), arr.ind = TRUE)
  cordata <- data.table(col1=colnames(temp_cor)[temp_corIndex[,1]],col2=colnames(temp_cor)[temp_corIndex[,2]])
  
  if(nrow(cordata)>0) {
    drop_cor_vars <- c()
    for (i in 1:nrow(cordata)) {
      if(cordata[i][,1,with=F]=="group"|cordata[i][,2,with=F]=="group"){
        if(cordata[i][,1,with=F]=="group") drop_cor_vars <- c(drop_cor_vars,as.character(cordata[i][,2,with=F]))
        if(cordata[i][,2,with=F]=="group") drop_cor_vars <- c(drop_cor_vars,as.character(cordata[i][,1,with=F]))
      } else {
        if (summary(model_glm_2)$coefficients[which(rownames(summary(model_glm_2)$coefficients)==as.character(cordata[i][,1,with=F])),4] > summary(model_glm_2)$coefficients[which(rownames(summary(model_glm_2)$coefficients)==as.character(cordata[i][,2,with=F])),4]) {
          drop_cor_vars <- c(drop_cor_vars,as.character(cordata[i][,1,with=F]))
        } else {
          drop_cor_vars <- c(drop_cor_vars,as.character(cordata[i][,2,with=F]))
        }
      }
    }
    vars_2 <- setdiff(vars_2,drop_cor_vars)
  }
}

#######################################
#------------ Final Model ------------#
#######################################

# set the formula for the final model
Formula_final <- paste0(paste0(y_var,"~ ",paste0(vars_2,collapse=" + ")," ")," + group + offset(log(Dol_per_Trip_PRE_P1+1))")

# run model
dolocc_final_model <-  glm(Formula_final,data=finaldata_dolocc,family=Gamma(link="log"))
summary(dolocc_final_model)
rm(model_glm_2)

# final model
# final cleaning of model based on signs
pvalues_2 <- summary(dolocc_final_model)$coefficients[,4][-1]
keep_covariate_2 <- names(pvalues_2[pvalues_2 < pvalue_lvl])
coeffs <- sign(summary(dolocc_final_model)$coefficients[,1])
finalvars <- c();finalvars1 <- c()
for (i in negativevars) if (i %in% names(coeffs[coeffs<0])) finalvars1 <- c(finalvars1,i)
for (j in positivevars) if (j %in% names(coeffs[coeffs>0])) finalvars1 <- c(finalvars1,j)
for (l in neutralvars) finalvars1 <- c(finalvars1,l)
for (k in finalvars1) if (k %in% keep_covariate_2) finalvars <- c(finalvars,k)

#set the formula for the final model
Formula_final <-paste0(paste0(y_var,"~ ",paste0(finalvars,collapse=" + ")," ")," + group + offset(log(Dol_per_Trip_PRE_P1+1))")
dolocc_final_model <-  glm(Formula_final,data=finaldata_dolocc,family=Gamma(link="log"))
summary(dolocc_final_model)

options(warn=0)

dolocc_random_campaigns <- intersect(random_campaigns,names(finaldata_dolocc))

if(length(dolocc_random_campaigns)>0){
  dolocc_random_campaigns_factors <- paste0("+(0+group|",dolocc_random_campaigns,")")
} else {
  dolocc_random_campaigns_factors <- NULL
}
random_demos_factors <- paste0( "+(0+group|",random_demos,")" )

##################################################################################################################

# run glmer model
dolocc.mixed.start.time <- Sys.time()
Formula_final <- paste0(c(paste0(paste0(y_var,"~ ",paste0(finalvars,collapse=" + ")," "),"+ offset(log(Dol_per_Trip_PRE_P1+1)) + group"),dolocc_random_campaigns_factors),collapse='')
dolocc_final_model <- glmer(Formula_final,data=finaldata_dolocc,family=Gamma(link="log"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
dolocc.mixed.model.execution.time <- Sys.time()-dolocc.mixed.start.time
out_resid <- order(residuals(dolocc_final_model),decreasing = T)
finaldata_dolocc <- as.data.table(finaldata_dolocc)
dolocc_total_outliers_panid <- finaldata_dolocc[out_resid,panid]
segment_panid <- data_NB_NE_B[,panid]
dolocc_outliers_panid_segm <- intersect(dolocc_total_outliers_panid,segment_panid)

# remove outliers if needed (negative estimation for exposure)
dolocc_outliers_panid_to_remove <- c()
dolocc_outliers_panid_segm_new <- c()
dolocc_outliers_panid_to_remove_threshold <- c()

flag <- c()
coef_flag <- c()
for(i in setdiff(levels(finaldata_dolocc$group),"0")) flag <- append(flag,paste0('group',i))

for(j in flag){
  coef_flag <- append(coef_flag,coef(summary(dolocc_final_model))[j,"Estimate"])
}

if(length(dolocc_random_campaigns)>0){
  min_random_coeff <- min(unlist(lapply(ranef(dolocc_final_model)[dolocc_random_campaigns],FUN=min)))
  max_random_coeff <- max(unlist(lapply(ranef(dolocc_final_model)[dolocc_random_campaigns],FUN=max)))
} else {
  min_random_coeff <- 0
}

if(min_random_coeff < -0.05){
  out_resid_positive <- order(residuals(dolocc_final_model),decreasing = T)
  out_resid_negative <- order(residuals(dolocc_final_model),decreasing = F)
  dolocc_outliers_panid_positive_extreme_panid <- finaldata_dolocc[out_resid_positive,panid]
  dolocc_outliers_panid_negative_extreme_panid <- finaldata_dolocc[out_resid_negative,panid]
  for(i in 1:length(dolocc_random_campaigns)){
    min_random_coeff <- min(unlist(lapply(ranef(dolocc_final_model)[i],FUN=min)))
    if(min_random_coeff < -0.05){
      max_random_coeff <- max(unlist(lapply(ranef(dolocc_final_model)[i],FUN=max)))
      random_frame <- as.data.frame(ranef(dolocc_final_model)[i])
      random_table <- as.data.table(random_frame)
      random_table[,level:=row.names(random_frame)]
      negative_extreme_levels <- random_table[eval(parse(text=paste0(names(ranef(dolocc_final_model)[i]),'.group1')))==min_random_coeff,]$level
      positive_extreme_levels <- random_table[eval(parse(text=paste0(names(ranef(dolocc_final_model)[i]),'.group1')))== max_random_coeff]$level
      negative_extreme_panid <- finaldata_dolocc[eval(parse(text=paste0(names(ranef(dolocc_final_model)[i])))) %in% negative_extreme_levels,panid] # -13%
      positive_extreme_panid  <- finaldata_dolocc[eval(parse(text=paste0(names(ranef(dolocc_final_model)[i])))) %in% positive_extreme_levels,panid] # +12%
      dolocc_outliers_panid_segm1 <- intersect(dolocc_outliers_panid_negative_extreme_panid,negative_extreme_panid);dolocc_outliers_panid_segm1 <- dolocc_outliers_panid_segm1[1:round(0.02*(length(negative_extreme_panid)))]
      if(max_random_coeff > 0.05) {
        dolocc_outliers_panid_segm2 <- intersect(dolocc_outliers_panid_positive_extreme_panid,positive_extreme_panid)
        dolocc_outliers_panid_segm2 <- dolocc_outliers_panid_segm2[1:round(0.02*(length(positive_extreme_panid)))]
      } else {
        dolocc_outliers_panid_segm2 <- c()
      }
      dolocc_outliers_panid_segm_new <- c(dolocc_outliers_panid_segm1,dolocc_outliers_panid_segm2)
      finaldata_dolocc <- finaldata_dolocc[!finaldata_dolocc$panid %in% dolocc_outliers_panid_segm_new,]
    }
  }
  dolocc_final_model <- glmer(Formula_final,data=finaldata_dolocc,family=Gamma(link="log"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
  min_random_coeff <- min(unlist(lapply(ranef(dolocc_final_model)[dolocc_random_campaigns],FUN=min)))
  if(min_random_coeff < -0.05){
    for(i in 1:length(dolocc_random_campaigns)){
      min_random_coeff <- min(unlist(lapply(ranef(dolocc_final_model)[i],FUN=min)))
      if(min_random_coeff < -0.05){
        max_random_coeff <- max(unlist(lapply(ranef(dolocc_final_model)[i],FUN=max)))
        random_frame <- as.data.frame(ranef(dolocc_final_model)[i])
        random_table <- as.data.table(random_frame)
        random_table[,level:=row.names(random_frame)]
        negative_extreme_levels <- random_table[eval(parse(text=paste0(names(ranef(dolocc_final_model)[i]),'.group1')))==min_random_coeff,]$level
        positive_extreme_levels <- random_table[eval(parse(text=paste0(names(ranef(dolocc_final_model)[i]),'.group1')))== max_random_coeff]$level
        negative_extreme_panid <- finaldata_dolocc[eval(parse(text=paste0(names(ranef(dolocc_final_model)[i])))) %in% negative_extreme_levels,panid] # -13%
        positive_extreme_panid  <- finaldata_dolocc[eval(parse(text=paste0(names(ranef(dolocc_final_model)[i])))) %in% positive_extreme_levels,panid] # +12%
        dolocc_outliers_panid_segm1 <- intersect(dolocc_outliers_panid_negative_extreme_panid,negative_extreme_panid);dolocc_outliers_panid_segm1 <- dolocc_outliers_panid_segm1[1:round(0.05*(length(negative_extreme_panid)))]
        if(max_random_coeff > 0.05) {
          dolocc_outliers_panid_segm2 <- intersect(dolocc_outliers_panid_positive_extreme_panid,positive_extreme_panid)
          dolocc_outliers_panid_segm2 <- dolocc_outliers_panid_segm2[1:round(0.05*(length(positive_extreme_panid)))]
        } else {
          dolocc_outliers_panid_segm2 <- c()
        }
        dolocc_outliers_panid_segm_new <- c(dolocc_outliers_panid_segm1,dolocc_outliers_panid_segm2)
        finaldata_dolocc <- finaldata_dolocc[!finaldata_dolocc$panid %in% dolocc_outliers_panid_segm_new,]
      }
    }
    dolocc_final_model <- glmer(Formula_final,data=finaldata_dolocc,family=Gamma(link="log"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
    min_random_coeff <- min(unlist(lapply(ranef(dolocc_final_model)[dolocc_random_campaigns],FUN=min)))
  }
  if(min_random_coeff < -0.05){
    for(i in 1:length(dolocc_random_campaigns)){
      min_random_coeff <- min(unlist(lapply(ranef(dolocc_final_model)[i],FUN=min)))
      if(min_random_coeff < -0.05){
        max_random_coeff <- max(unlist(lapply(ranef(dolocc_final_model)[i],FUN=max)))
        random_frame <- as.data.frame(ranef(dolocc_final_model)[i])
        random_table <- as.data.table(random_frame)
        random_table[,level:=row.names(random_frame)]
        negative_extreme_levels <- random_table[eval(parse(text=paste0(names(ranef(dolocc_final_model)[i]),'.group1')))==min_random_coeff,]$level
        positive_extreme_levels <- random_table[eval(parse(text=paste0(names(ranef(dolocc_final_model)[i]),'.group1')))== max_random_coeff]$level
        negative_extreme_panid <- finaldata_dolocc[eval(parse(text=paste0(names(ranef(dolocc_final_model)[i])))) %in% negative_extreme_levels,panid] # -13%
        positive_extreme_panid  <- finaldata_dolocc[eval(parse(text=paste0(names(ranef(dolocc_final_model)[i])))) %in% positive_extreme_levels,panid] # +12%        dolocc_outliers_panid_segm1 <- intersect(dolocc_outliers_panid_negative_extreme_panid,negative_extreme_panid);dolocc_outliers_panid_segm1 <- dolocc_outliers_panid_segm1[1:round(0.10*(length(negative_extreme_panid)))]
        if(max_random_coeff > 0.05) {
          dolocc_outliers_panid_segm2 <- intersect(dolocc_outliers_panid_positive_extreme_panid,positive_extreme_panid)
          dolocc_outliers_panid_segm2 <- dolocc_outliers_panid_segm2[1:round(0.10*(length(positive_extreme_panid)))]
        } else {
          dolocc_outliers_panid_segm2 <- c()
        }
        dolocc_outliers_panid_segm_new <- c(dolocc_outliers_panid_segm1,dolocc_outliers_panid_segm2)
        finaldata_dolocc <- finaldata_dolocc[!finaldata_dolocc$panid %in% dolocc_outliers_panid_segm_new,]
      }
    }
    dolocc_final_model <- glmer(Formula_final,data=finaldata_dolocc,family=Gamma(link="log"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
    min_random_coeff <- min(unlist(lapply(ranef(dolocc_final_model)[dolocc_random_campaigns],FUN=min)))
  }
}

if(length(dolocc_random_campaigns)>0){
  min_random_coeff <- min(unlist(lapply(ranef(dolocc_final_model)[dolocc_random_campaigns],FUN=min)))
  max_random_coeff <- max(unlist(lapply(ranef(dolocc_final_model)[dolocc_random_campaigns],FUN=max)))
} else {
  min_random_coeff <- 0
}

coef_flag <- c()
pvalue_flag <- c()
for(j in flag){
  coef_flag <- append(coef_flag,coef(summary(dolocc_final_model))[j,"Estimate"])
  pvalue_flag <- append(pvalue_flag,coef(summary(dolocc_final_model))[j,"Pr(>|z|)"])
}

# drop outliers in case of high exposure estimation (in case we want a lower group1 coef to meet business requirements)
# segment_panid <- dol_reduction[,panid]
# outliers_panid <- intersect(dolocc_total_outliers_panid,segment_panid)
# dolocc_outliers_panid_to_remove_threshold <- outliers_panid[c(1:round(0.10*(length(outliers_panid))))]
# finaldata_dolocc <- finaldata_dolocc[!finaldata_dolocc$panid %in% dolocc_outliers_panid_to_remove_threshold,]
# dolocc_final_model <- glmer(Formula_final,data=finaldata_dolocc,family=Gamma(link="log"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))

if(min(coef_flag) + min_random_coeff  < 0| max(pvalue_flag)>0.20){
  cat('\n')
  cat("removing outliers ...\nEstimated Iteration Execution ");print(dolocc.mixed.model.execution.time)
  cat("\nIteration 1 ... Start Time: ");print(Sys.time())
  dolocc_outliers_panid_to_remove <- dolocc_outliers_panid_segm[1:round(0.025*(nrow(data_NB_NE_B)))]
  finaldata_dolocc <- finaldata_dolocc[!finaldata_dolocc$panid %in% dolocc_outliers_panid_to_remove,]
  dolocc_final_model <- glmer(Formula_final,data=finaldata_dolocc,family=Gamma(link="log"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
  coef_flag <- c()
  pvalue_flag <- c()
  for(j in flag){
    coef_flag <- append(coef_flag,coef(summary(dolocc_final_model))[j,"Estimate"])
    pvalue_flag <- append(pvalue_flag,coef(summary(dolocc_final_model))[j,"Pr(>|z|)"])
  }
  min_random_coeff <- min(unlist(lapply(ranef(dolocc_final_model)[dolocc_random_campaigns],FUN=min)))
  if(min(coef_flag) + min_random_coeff  < 0| max(pvalue_flag)>0.20){
    dolocc_outliers_panid_to_remove <- dolocc_outliers_panid_segm[1:round(0.05*(nrow(data_NB_NE_B)))]  
    finaldata_dolocc <- finaldata_dolocc[!finaldata_dolocc$panid %in% dolocc_outliers_panid_to_remove,]
    cat("\nIteration 2 ... Start Time: ");print(Sys.time())
    dolocc_final_model <- glmer(Formula_final,data=finaldata_dolocc,family=Gamma(link="log"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
    coef_flag <- c()
    pvalue_flag <- c()
    for(j in flag){
      coef_flag <- append(coef_flag,coef(summary(dolocc_final_model))[j,"Estimate"])
      pvalue_flag <- append(pvalue_flag,coef(summary(dolocc_final_model))[j,"Pr(>|z|)"])
    }
    min_random_coeff <- min(unlist(lapply(ranef(dolocc_final_model)[dolocc_random_campaigns],FUN=min)))
    if(min(coef_flag) + min_random_coeff < 0| max(pvalue_flag)>0.20 ){
      dolocc_outliers_panid_to_remove <- dolocc_outliers_panid_segm[1:round(0.10*(nrow(data_NB_NE_B)))]  
      finaldata_dolocc <- finaldata_dolocc[!finaldata_dolocc$panid %in% dolocc_outliers_panid_to_remove,]
      cat("\nIteration 3 ... Start Time: ");print(Sys.time())
      dolocc_final_model <- glmer(Formula_final,data=finaldata_dolocc,family=Gamma(link="log"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
      coef_flag <- c()
      pvalue_flag <- c()
      for(j in flag){
        coef_flag <- append(coef_flag,coef(summary(dolocc_final_model))[j,"Estimate"])
        pvalue_flag <- append(pvalue_flag,coef(summary(dolocc_final_model))[j,"Pr(>|z|)"])
      }
      min_random_coeff <- min(unlist(lapply(ranef(dolocc_final_model)[dolocc_random_campaigns],FUN=min)))
      if(min(coef_flag) + min_random_coeff  < 0| max(pvalue_flag)>0.20){
        dolocc_outliers_panid_to_remove <- dolocc_outliers_panid_segm[1:round(0.15*(nrow(data_NB_NE_B)))]  
        finaldata_dolocc <- finaldata_dolocc[!finaldata_dolocc$panid %in% dolocc_outliers_panid_to_remove,]
        cat("\nFinal Iteration ... Start Time: ");print(Sys.time())
        dolocc_final_model <- glmer(Formula_final,data=finaldata_dolocc,family=Gamma(link="log"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
      }
    }
  }
}

write.csv(as.data.frame(summary(dolocc_final_model)$coefficients),file=paste0(initial_DollOcc_modelling_output,"DolOcc_fixed_effects.csv"))

# create output for random effects
randoms <- as.data.frame(augment.ranef.mer(ranef(dolocc_final_model,condVar=TRUE)))
random.ef_temp.report <- randoms[,!names(randoms) %in% c("qq")]
write.csv(random.ef_temp.report,file=paste0(initial_DollOcc_modelling_output,"DolOcc_random_effects_pvalue.csv"),row.names=FALSE)

for(i in dolocc_random_campaigns){
  random.ef_temp <- as.data.frame(ranef(dolocc_final_model)[i])
  random.coef_temp <- as.data.table(random.ef_temp[2],keep.rownames=TRUE)
  random.coef <- random.coef_temp[,label:=names(random.coef_temp)[2]]
  write.table(random.coef,file=paste0(initial_DollOcc_modelling_output,'DolOcc_random_effects_coeffs.csv'), quote = FALSE,append    = i > 1, 
              sep= ",", row.names = FALSE,col.names=FALSE)
}

for(i in dolocc_random_campaigns){
  random.ef_temp <- as.data.frame(se.ranef(dolocc_final_model)[i])
  colnames(random.ef_temp)[2] <- paste0(i,' S.E.')
  options(warn=-1);write.table(random.ef_temp[2],file=paste0(initial_DollOcc_modelling_output,'DolOcc_SE_random_effects.csv'), quote = FALSE,append    = i > 1, 
                               sep= ",", row.names = FALSE);options(warn=0)
}

dolocc_random_demos <- c()
random_groupe <- c(dolocc_random_demos,dolocc_random_campaigns)

if(length(dolocc_random_demos)>0){
  random_group_demos <- data.frame(random_demos)
  names.random_group_demos <- c("class")
  setnames(random_group_demos,names.random_group_demos)
  random_group_demos$exposed <- 0
} else {
  random_group_demos <- c()
}

random_group_campaigns <- data.frame(dolocc_random_campaigns)
names.random_group_campaigns <- c("class")
setnames(random_group_campaigns,names.random_group_campaigns)
random_group_campaigns$exposed <- 1

random_group_exposed <- rbind( random_group_campaigns,random_group_demos)

random_group_data <- data.frame()
for(i in random_groupe){
  random.ef_temp <- as.data.frame(ranef(dolocc_final_model)[i])
  random.coef_temp <- as.data.table(random.ef_temp,keep.rownames=TRUE)
  random.coef <- random.coef_temp[,class:=i]
  names.random.coef <- c("level","group0","group1","class")
  setnames(random.coef ,names.random.coef)
  random.coef$row <- row.names(random.coef)
  random_group_data <-  rbind( random.coef,random_group_data)
}

random_group_SEdata <- data.frame()
for(i in random_groupe){
  random.sef_temp <- as.data.frame(se.ranef(dolocc_final_model)[i])
  random.secoef_temp <- as.data.table(random.sef_temp,keep.rownames=TRUE)
  random.secoef <- random.secoef_temp[,class:=i]
  names.random.secoef <- c("levelSE","SE_group0","SE_group1","class")
  setnames(random.secoef ,names.random.secoef)
  random.secoef$row <- row.names(random.secoef)
  random_group_SEdata <-  rbind( random.secoef,random_group_SEdata)
}

final_random <- merge(random_group_data,random_group_SEdata, by=c("class","row"))
final_random <- merge(final_random,random_group_exposed, by=c("class"))
View(final_random)
write.csv(final_random,file=paste0(initial_DollOcc_modelling_output,'DolOcc_random_effects.csv'), row.names = FALSE)

# create a txt file with the summary of the glmer model 
logsavefname <- "DolOcc_Initial_Results.txt"
sink(paste0(initial_DollOcc_modelling_output,logsavefname))
print(Sys.time()-dolocc.start.time)
print(summary(dolocc_final_model))
print(ranef(dolocc_final_model))
sink()

for(i in dolocc_random_campaigns){
  pdf(paste0(initial_DollOcc_modelling_output,"DolOcc_Caterpillar_Plots ",i,".pdf"),width=7,height=5)
  print(ggCaterpillar(ranef(dolocc_final_model, condVar=TRUE)[i]))
  dev.off()
}

# save DolOcc final dataset and final model
dolocc_Formula_final <- Formula_final
dolocc_outliers_panid_to_remove <- c(dolocc_outliers_panid_to_remove,dolocc_outliers_panid_segm_new,dolocc_outliers_panid_to_remove_threshold)
rm(finaldata_dolocc_num,temp_cor,cordata)

# save data in .Rdata file
save(finaldata_dolocc,dolocc_Formula_final,dolocc_final_model,dolocc_outliers_panid_to_remove,file=paste0(output,"saved_data/","DolOcc_initial_model_data.Rdata"))

DolOcc.Execution.Time <- Sys.time()-dolocc.start.time
cat(paste0("DolOcc.Execution.Time: ", DolOcc.Execution.Time))

######################################
#--------------OCCASION--------------#
######################################

occ.start.time <- Sys.time()

# set response variable
y_var <- "Trps_POS_P1"

# keep only the buyers
finaldata_occ <- initial_data[Buyer_Pos_P1=="1",]

# drop variables with only one level
indices <- sapply(finaldata_occ,function(x) length(unique(x))==1)
finaldata_occ <- finaldata_occ[,!indices,with=F]

#######################################
#-------- Variable Selection ---------#
#######################################

# remove dolocc outliers
finaldata_occ <- as.data.frame(finaldata_occ)
if(exists('dolocc_outliers_panid_to_remove', envir = environment())){
  finaldata_occ <- finaldata_occ[, !(names(finaldata_occ) %in% c("Dol_per_Trip_POS_P1","Nonbuyer_Pre_P1","Buyer_Pre_P1","Dol_per_Trip_PRE_P1"))]
  finaldata_occ <- finaldata_occ[!finaldata_occ$panid %in% dolocc_outliers_panid_to_remove,]
}else{
  finaldata_occ <- finaldata_occ[, !(names(finaldata_occ) %in% c("Dol_per_Trip_POS_P1","Nonbuyer_Pre_P1","Buyer_Pre_P1","Dol_per_Trip_PRE_P1"))]
}

# set initial model formula
Formula_fixed <- paste0(paste0(y_var,"~ ",paste0(names(finaldata_occ[,!names(finaldata_occ)%in%c(y_var,"Trps_PRE_P1","panid",random_demos,random_campaigns,scoring_vars)]),collapse=" + ")," "),"+ offset(log(Trps_PRE_P1+1))")
options(warn=-1)

# run glm model
model_glm_total <- glm(Formula_fixed,data=finaldata_occ,family=poisson(link="log"))
summary(model_glm_total)

##### Variable selection - Condition 1 #####
# Remove pvalues greater than 0.7 and rerun the glm model
pval_excluded_vars <- c(rownames(as.data.frame(which(summary(model_glm_total)$coefficient[,4]>0.7))),setdiff(names(model_glm_total$coefficients),rownames(summary(model_glm_total)$coefficients)))
finaldata_occ <- finaldata_occ[, !names(finaldata_occ) %in% c(pval_excluded_vars)] 

# re-run glm model
Formula_fixed <- paste0(paste0(y_var,"~ ",paste0(names(finaldata_occ[,!names(finaldata_occ)%in%c(y_var,"Trps_PRE_P1","panid",random_demos,random_campaigns,scoring_vars)]),collapse=" + ")," "),"+ offset(log(Trps_PRE_P1+1))")
model_glm_condition1 <- glm(Formula_fixed,data=finaldata_occ,family=poisson(link="log"))
summary(model_glm_condition1)
rm(model_glm_total)

##### Variable selection - Condition 2 #####
# Remove variables that have z value lower than 2 and VIF greater than 15

# get variables with z value lower than 2
z_excluded_vars <- rownames(as.data.frame(which(abs(summary(model_glm_condition1)$coefficient[,3])<1.96)))

# get the variables with VIF greater than 15
multicol <- alias(model_glm_condition1)
multicols_toremove <- rownames(as.data.frame(multicol$Complete))

if (length(multicols_toremove)>0){
  finaldata_occ <- finaldata_occ[ , !names(finaldata_occ) %in% multicols_toremove] 
  Formula_fixed <-  paste0(paste0(y_var,"~ ",paste0(names(finaldata_occ[,!names(finaldata_occ)%in%c(y_var,"panid",random_demos,random_campaigns)]),collapse=" + ")," "),"+ offset(log(Trps_PRE_P1+1)) ")
  model_glm_new <- glm(Formula_fixed,data=finaldata_occ,family=poisson(link="log"))
} else {
  model_glm_new <- model_glm_condition1
}

vif_output <- (vif(model_glm_new))
vif_to_remove <- names(vif_output[vif_output > 15])

# final vars to exclude under condition 2
vars_condition2 <- intersect(z_excluded_vars,vif_to_remove)

# remove the variables under the above conditions
finaldata_occ <- finaldata_occ[ , !names(finaldata_occ) %in% c(vars_condition2,pval_excluded_vars)] 
Formula_fixed <- paste0(paste0(y_var,"~ ",paste0(names(finaldata_occ[,!names(finaldata_occ)%in%c(y_var,"panid","Trps_PRE_P1",random_demos,random_campaigns,scoring_vars)]),collapse=" + ")," "),"+ offset(log(Trps_PRE_P1+1))")

# run model
model_glm <- glm(Formula_fixed,data=finaldata_occ,family=poisson(link="log"))
summary(model_glm)
rm(model_glm_condition1)

# p-values and signs checks
neutralvars <- setdiff(names(finaldata_occ),c(positivevars,negativevars))
pvalues <- summary(model_glm)$coefficients[,4][-1]
keep_covariate <- names(pvalues[pvalues<pvalue_lvl])
coeffs <- sign(summary(model_glm)$coefficients[,1])
varstokeep <- c();initialvars <- c()
for (i in negativevars) if (i %in% names(coeffs[coeffs<0])) varstokeep <- c(varstokeep,i)
for (j in positivevars) if (j %in% names(coeffs[coeffs>0])) varstokeep <- c(varstokeep,j)
for (l in neutralvars) varstokeep <- c(varstokeep,l)
for (k in varstokeep) if (k %in% keep_covariate) initialvars <- c(initialvars,k)

#set the formula for the second model after cleaning the first model
Formula_final <- paste0(paste0(y_var,"~ ",paste0(initialvars,collapse=" + ")," "),"+ group+ offset(log(Trps_PRE_P1+1))")

model_glm_2 <-  glm(Formula_final,data=finaldata_occ,family=poisson(link="log"))
summary(model_glm_2)
rm(model_glm)

# final cleaning of second model based on signs and pvalues
pvalues_2 <- summary(model_glm_2)$coefficients[,4][-1]
keep_covariate_2 <- names(pvalues_2[pvalues_2 < pvalue_lvl])
coeffs_2 <- sign(summary(model_glm_2)$coefficients[,1])
varstokeep <- c();vars_2 <- c()
for (i in negativevars) if (i %in% names(coeffs_2[coeffs_2 < 0])) varstokeep <- c(varstokeep,i)
for (j in positivevars) if (j %in% names(coeffs_2[coeffs_2 > 0])) varstokeep <- c(varstokeep,j)
for (l in neutralvars) varstokeep <- c(varstokeep,l)
for (k in varstokeep) if (k %in% keep_covariate_2) vars_2 <- c(vars_2,k)

# final correlation check (find pair vars with correlation greater then 0.8 and drop the one with bigger pvalue) #### Maybe 0.6!
if(length(vars_2)>1){
  finaldata_occ_num <- as.data.table(finaldata_occ[,names(finaldata_occ) %in% vars_2])
  t1 <- as.data.frame(finaldata_occ_num[,sapply(finaldata_occ_num,is.numeric)])
  finaldata_occ_num <- as.data.frame(finaldata_occ_num)
  finaldata_occ_num <- finaldata_occ_num[,c(which(t1==T))]
  temp_cor <- as.matrix(cor(finaldata_occ_num))
  temp_cor[lower.tri(temp_cor, diag = T)] <- NA
  temp_corIndex <- which(temp_cor > 0.8 |temp_cor < (-0.8), arr.ind = TRUE)
  cordata <- data.table(col1=colnames(temp_cor)[temp_corIndex[,1]],col2=colnames(temp_cor)[temp_corIndex[,2]])
  
  if(nrow(cordata)>0) {
    drop_cor_vars <- c()
    for (i in 1:nrow(cordata)) {
      if(cordata[i][,1,with=F]=="group"|cordata[i][,2,with=F]=="group"){
        if(cordata[i][,1,with=F]=="group") drop_cor_vars <- c(drop_cor_vars,as.character(cordata[i][,2,with=F]))
        if(cordata[i][,2,with=F]=="group") drop_cor_vars <- c(drop_cor_vars,as.character(cordata[i][,1,with=F]))
      } else {
        if (summary(model_glm_2)$coefficients[which(rownames(summary(model_glm_2)$coefficients)==as.character(cordata[i][,1,with=F])),4] > summary(model_glm_2)$coefficients[which(rownames(summary(model_glm_2)$coefficients)==as.character(cordata[i][,2,with=F])),4]) {
          drop_cor_vars <- c(drop_cor_vars,as.character(cordata[i][,1,with=F]))
        } else {
          drop_cor_vars <- c(drop_cor_vars,as.character(cordata[i][,2,with=F]))
        }
      }
    }
    vars_2 <- setdiff(vars_2,drop_cor_vars)
  }
}

#######################################
#------------ Final Model ------------#
#######################################

#set the formula for the final model
Formula_final <-paste0( paste0(y_var,"~ ",paste0(vars_2,collapse=" + ")," "),"+ group + offset(log(Trps_PRE_P1+1))")
occ_final_model <-  glm(Formula_final,data=finaldata_occ,family=poisson(link="log"))
summary(occ_final_model)
rm(model_glm_2)

# final model
# final cleaning of model based on signs
pvalues_2 <- summary(occ_final_model)$coefficients[,4][-1]
keep_covariate_2 <- names(pvalues_2[pvalues_2 < pvalue_lvl])
coeffs <- sign(summary(occ_final_model)$coefficients[,1])
finalvars <- c();finalvars1 <- c()
for (i in negativevars) if (i %in% names(coeffs[coeffs<0])) finalvars <- c(finalvars,i)
for (j in positivevars) if (j %in% names(coeffs[coeffs>0])) finalvars <- c(finalvars,j)
for (l in neutralvars) if (l %in% keep_covariate_2) finalvars <- c(finalvars,l)
for (l in neutralvars) finalvars1 <- c(finalvars1,l)
for (k in finalvars1) if (k %in% keep_covariate_2) finalvars <- c(finalvars,k)

#set final model
Formula_final <- paste0(paste0(y_var,"~ ",paste0(finalvars,collapse=" + ")," "),"+ group + offset(log(Trps_PRE_P1+1))")
occ_final_model <-  glm(Formula_final,data=finaldata_occ,family=poisson(link="log"))
summary(occ_final_model)

options(warn=0)

finaldata_occ <- droplevels(finaldata_occ)
occ_random_campaigns <- intersect(random_campaigns,names(finaldata_occ))

if(length(occ_random_campaigns)>0){
  occ_random_campaigns_factors <- paste0("+(0+group|",occ_random_campaigns,")")
} else {
  occ_random_campaigns_factors <- NULL
}
random_demos_factors <- paste0( "+(0+group|",random_demos,")" )

##################################################################################################################

# run glmer model
occ.mixed.start.time <- Sys.time()
Formula_final <- paste0(c(paste0(paste0(y_var,"~ ",paste0(finalvars,collapse=" + ")," "),"+ offset(log(Trps_PRE_P1+1)) + group"),random_demos_factors,occ_random_campaigns_factors),collapse='')
occ_final_model <- glmer(Formula_final,data=finaldata_occ,family=poisson(link="log"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
occ.mixed.model.execution.time <- Sys.time()-occ.mixed.start.time
out_resid <- order(residuals(occ_final_model),decreasing = T)
finaldata_occ <- as.data.table(finaldata_occ)
occ_total_outliers_panid <- finaldata_occ[out_resid,panid]
segment_panid <- data_NB_NE_B[,panid]
occ_outliers_panid_segm <- intersect(occ_total_outliers_panid,segment_panid)

# remove outliers if needed (negative estimation for exposure)
occ_outliers_panid_to_remove <- c()
occ_outliers_panid_segm_new <- c()
occ_outliers_panid_to_remove_threshold <- c()

flag <- c()
coef_flag <- c()
for(i in setdiff(levels(finaldata_occ$group),"0")) flag <- append(flag,paste0('group',i))

for(j in flag){
  coef_flag <- append(coef_flag,coef(summary(occ_final_model))[j,"Estimate"])
}

if(length(occ_random_campaigns)>0){
  min_random_coeff <- min(unlist(lapply(ranef(occ_final_model)[occ_random_campaigns],FUN=min)))
  max_random_coeff <- max(unlist(lapply(ranef(occ_final_model)[occ_random_campaigns],FUN=max)))
} else {
  min_random_coeff <- 0
}

if(min_random_coeff < -0.05){
  out_resid_positive <- order(residuals(occ_final_model),decreasing = T)
  out_resid_negative <- order(residuals(occ_final_model),decreasing = F)
  occ_outliers_panid_positive_extreme_panid <- finaldata_occ[out_resid_positive,panid]
  occ_outliers_panid_negative_extreme_panid <- finaldata_occ[out_resid_negative,panid]
  for(i in 1:length(occ_random_campaigns)){
    min_random_coeff <- min(unlist(lapply(ranef(occ_final_model)[i],FUN=min)))
    if(min_random_coeff < -0.05){
      max_random_coeff <- max(unlist(lapply(ranef(occ_final_model)[i],FUN=max)))
      random_frame <- as.data.frame(ranef(occ_final_model)[i])
      random_table <- as.data.table(random_frame)
      random_table[,level:=row.names(random_frame)]
      negative_extreme_levels <- random_table[eval(parse(text=paste0(names(ranef(occ_final_model)[i]),'.group1')))==min_random_coeff,]$level
      positive_extreme_levels <- random_table[eval(parse(text=paste0(names(ranef(occ_final_model)[i]),'.group1')))== max_random_coeff]$level
      negative_extreme_panid <- finaldata_occ[eval(parse(text=paste0(names(ranef(occ_final_model)[i])))) %in% negative_extreme_levels,panid] # -13%
      positive_extreme_panid  <- finaldata_occ[eval(parse(text=paste0(names(ranef(occ_final_model)[i])))) %in% positive_extreme_levels,panid] # +12%
      occ_outliers_panid_segm1 <- intersect(occ_outliers_panid_negative_extreme_panid,negative_extreme_panid);occ_outliers_panid_segm1 <- occ_outliers_panid_segm1[1:round(0.02*(length(negative_extreme_panid)))]
      if(max_random_coeff > 0.05) {
        occ_outliers_panid_segm2 <- intersect(occ_outliers_panid_positive_extreme_panid,positive_extreme_panid)
        occ_outliers_panid_segm2 <- occ_outliers_panid_segm2[1:round(0.02*(length(positive_extreme_panid)))]
      } else {
        occ_outliers_panid_segm2 <- c()
      }
      occ_outliers_panid_segm_new <- c(occ_outliers_panid_segm1,occ_outliers_panid_segm2)
      finaldata_occ <- finaldata_occ[!finaldata_occ$panid %in% occ_outliers_panid_segm_new,]
    }
  }
  occ_final_model <- glmer(Formula_final,data=finaldata_occ,family=poisson(link="log"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
  min_random_coeff <- min(unlist(lapply(ranef(occ_final_model)[occ_random_campaigns],FUN=min)))
  if(min_random_coeff < -0.05){
    for(i in 1:length(occ_random_campaigns)){
      min_random_coeff <- min(unlist(lapply(ranef(occ_final_model)[i],FUN=min)))
      if(min_random_coeff < -0.05){
        max_random_coeff <- max(unlist(lapply(ranef(occ_final_model)[i],FUN=max)))
        random_frame <- as.data.frame(ranef(occ_final_model)[i])
        random_table <- as.data.table(random_frame)
        random_table[,level:=row.names(random_frame)]
        negative_extreme_levels <- random_table[eval(parse(text=paste0(names(ranef(occ_final_model)[i]),'.group1')))==min_random_coeff,]$level
        positive_extreme_levels <- random_table[eval(parse(text=paste0(names(ranef(occ_final_model)[i]),'.group1')))== max_random_coeff]$level
        negative_extreme_panid <- finaldata_occ[eval(parse(text=paste0(names(ranef(occ_final_model)[i])))) %in% negative_extreme_levels,panid] # -13%
        positive_extreme_panid  <- finaldata_occ[eval(parse(text=paste0(names(ranef(occ_final_model)[i])))) %in% positive_extreme_levels,panid] # +12%
        occ_outliers_panid_segm1 <- intersect(occ_outliers_panid_negative_extreme_panid,negative_extreme_panid);occ_outliers_panid_segm1 <- occ_outliers_panid_segm1[1:round(0.05*(length(negative_extreme_panid)))]
        if(max_random_coeff > 0.05) {
          occ_outliers_panid_segm2 <- intersect(occ_outliers_panid_positive_extreme_panid,positive_extreme_panid)
          occ_outliers_panid_segm2 <- occ_outliers_panid_segm2[1:round(0.05*(length(positive_extreme_panid)))]
        } else {
          occ_outliers_panid_segm2 <- c()
        }
        occ_outliers_panid_segm_new <- c(occ_outliers_panid_segm1,occ_outliers_panid_segm2)
        finaldata_occ <- finaldata_occ[!finaldata_occ$panid %in% occ_outliers_panid_segm_new,]
      }
    }
    occ_final_model <- glmer(Formula_final,data=finaldata_occ,family=poisson(link="log"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
    min_random_coeff <- min(unlist(lapply(ranef(occ_final_model)[occ_random_campaigns],FUN=min)))
  }
  if(min_random_coeff < -0.05){
    for(i in 1:length(occ_random_campaigns)){
      min_random_coeff <- min(unlist(lapply(ranef(occ_final_model)[i],FUN=min)))
      if(min_random_coeff < -0.05){
        max_random_coeff <- max(unlist(lapply(ranef(occ_final_model)[i],FUN=max)))
        random_frame <- as.data.frame(ranef(occ_final_model)[i])
        random_table <- as.data.table(random_frame)
        random_table[,level:=row.names(random_frame)]
        negative_extreme_levels <- random_table[eval(parse(text=paste0(names(ranef(occ_final_model)[i]),'.group1')))==min_random_coeff,]$level
        positive_extreme_levels <- random_table[eval(parse(text=paste0(names(ranef(occ_final_model)[i]),'.group1')))== max_random_coeff]$level
        negative_extreme_panid <- finaldata_occ[eval(parse(text=paste0(names(ranef(occ_final_model)[i])))) %in% negative_extreme_levels,panid] # -13%
        positive_extreme_panid  <- finaldata_occ[eval(parse(text=paste0(names(ranef(occ_final_model)[i])))) %in% positive_extreme_levels,panid] # +12%
        occ_outliers_panid_segm1 <- intersect(occ_outliers_panid_negative_extreme_panid,negative_extreme_panid);occ_outliers_panid_segm1 <- occ_outliers_panid_segm1[1:round(0.10*(length(negative_extreme_panid)))]
        if(max_random_coeff > 0.05) {
          occ_outliers_panid_segm2 <- intersect(occ_outliers_panid_positive_extreme_panid,positive_extreme_panid)
          occ_outliers_panid_segm2 <- occ_outliers_panid_segm2[1:round(0.10*(length(positive_extreme_panid)))]
        } else {
          occ_outliers_panid_segm2 <- c()
        }
        occ_outliers_panid_segm_new <- c(occ_outliers_panid_segm1,occ_outliers_panid_segm2)
        finaldata_occ <- finaldata_occ[!finaldata_occ$panid %in% occ_outliers_panid_segm_new,]
        occ_final_model <- glmer(Formula_final,data=finaldata_occ,family=Gamma(link="log"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
        min_random_coeff <- min(unlist(lapply(ranef(occ_final_model)[occ_random_campaigns],FUN=min)))
      }
    }
    occ_final_model <- glmer(Formula_final,data=finaldata_occ,family=poisson(link="log"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
    min_random_coeff <- min(unlist(lapply(ranef(occ_final_model)[occ_random_campaigns],FUN=min)))
  }
}

coef_flag <- c()
for(j in flag){
  coef_flag <- append(coef_flag,coef(summary(occ_final_model))[j,"Estimate"])
}

if(length(occ_random_campaigns)>0){
  min_random_coeff <- min(unlist(lapply(ranef(occ_final_model)[occ_random_campaigns],FUN=min)))
  max_random_coeff <- max(unlist(lapply(ranef(occ_final_model)[occ_random_campaigns],FUN=max)))
} else {
  min_random_coeff <- 0
}

# drop outliers in case of high exposure estimation (in case we want a lower group1 coef to meet business requirements)
# segment_panid <- occ_reduction[,panid]
# outliers_panid <- intersect(occ_total_outliers_panid,segment_panid)
# occ_outliers_panid_to_remove_threshold <- outliers_panid[c(1:round(0.10*(length(outliers_panid))))]
# finaldata_occ <- finaldata_occ[!finaldata_occ$panid %in% occ_outliers_panid_to_remove_threshold,]
# occ_final_model <- glmer(Formula_final,data=finaldata_occ,family=poisson(link="log"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))

if(min(coef_flag) + min_random_coeff < 0){
  cat("removing outliers ...\nEstimated Iteration Execution ");print(occ.mixed.model.execution.time)
  cat("\nIteration 1 ... Start Time: ");print(Sys.time())
  occ_outliers_panid_to_remove <- occ_outliers_panid_segm[1:round(0.025*(nrow(data_NB_NE_B)))]
  finaldata_occ <- finaldata_occ[!finaldata_occ$panid %in% occ_outliers_panid_to_remove,]
  occ_final_model <- glmer(Formula_final,data=finaldata_occ,family=poisson(link="log"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
  coef_flag <- c()
  for(j in flag){
    coef_flag <- append(coef_flag,coef(summary(occ_final_model))[j,"Estimate"])
  }
  min_random_coeff <- min(unlist(lapply(ranef(occ_final_model)[occ_random_campaigns],FUN=min)))
  if(min(coef_flag) + min_random_coeff < 0){
    occ_outliers_panid_to_remove <- occ_outliers_panid_segm[1:round(0.05*(nrow(data_NB_NE_B)))]  
    finaldata_occ <- finaldata_occ[!finaldata_occ$panid %in% occ_outliers_panid_to_remove,]
    cat("\nIteration 2 ... Start Time: ");print(Sys.time())
    occ_final_model <- glmer(Formula_final,data=finaldata_occ,family=poisson(link="log"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
    coef_flag <- c()
    for(j in flag){
      coef_flag <- append(coef_flag,coef(summary(occ_final_model))[j,"Estimate"])
    }
    min_random_coeff <- min(unlist(lapply(ranef(occ_final_model)[occ_random_campaigns],FUN=min)))
    if(min(coef_flag) + min_random_coeff < 0){
      occ_outliers_panid_to_remove <- occ_outliers_panid_segm[1:round(0.10*(nrow(data_NB_NE_B)))]  
      finaldata_occ <- finaldata_occ[!finaldata_occ$panid %in% occ_outliers_panid_to_remove,]
      cat("\nIteration 3 ... Start Time: ");print(Sys.time())
      occ_final_model <- glmer(Formula_final,data=finaldata_occ,family=poisson(link="log"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
      coef_flag <- c()
      for(j in flag){
        coef_flag <- append(coef_flag,coef(summary(occ_final_model))[j,"Estimate"])
      }
      min_random_coeff <- min(unlist(lapply(ranef(occ_final_model)[occ_random_campaigns],FUN=min)))
      if(min(coef_flag) + min_random_coeff < 0){
        occ_outliers_panid_to_remove <- occ_outliers_panid_segm[1:round(0.15*(nrow(data_NB_NE_B)))]  
        finaldata_occ <- finaldata_occ[!finaldata_occ$panid %in% occ_outliers_panid_to_remove,]
        cat("\nFinal Iteration ... Start Time: ");print(Sys.time())
        occ_final_model <- glmer(Formula_final,data=finaldata_occ,family=poisson(link="log"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
      }
    }
  }
}

occ_dolocc_outliers_panid_to_remove <- unique(c(dolocc_outliers_panid_to_remove,occ_outliers_panid_to_remove,occ_outliers_panid_segm_new,occ_outliers_panid_to_remove_threshold))

write.csv(as.data.frame(summary(occ_final_model)$coefficients),file=paste0(initial_Occ_modelling_output,"Occ_fixed_effects.csv"))

# create output for random effects
randoms <- as.data.frame(augment.ranef.mer(ranef(occ_final_model,condVar=TRUE)))
random.ef_temp.report <- randoms[,!names(randoms) %in% c("qq")]
write.csv(random.ef_temp.report,file=paste0(initial_Occ_modelling_output,"Occ_random_effects_pvalue.csv"),row.names=FALSE)

for(i in random_demos){
  random.ef_temp <- as.data.frame(ranef(occ_final_model)[i])
  random.coef_temp <- as.data.table(random.ef_temp,keep.rownames=TRUE)
  random.coef <- random.coef_temp[,class:=i]
  names(random.coef) <- c("","group0","group1","class")
  options(warn=-1);write.table(random.coef,file=paste0(initial_Occ_modelling_output,'Occ_random_demographics_effects.csv'), quote = FALSE,append    = i > 1, 
                               sep= ",", row.names = FALSE,col.names=ifelse(i %in% random_demos[1], TRUE, FALSE));options(warn=0)
}

for(i in occ_random_campaigns){
  random.ef_temp <- as.data.frame(ranef(occ_final_model)[i])
  random.coef_temp <- as.data.table(random.ef_temp[2],keep.rownames=TRUE)
  random.coef <- random.coef_temp[,label:=names(random.coef_temp)[2]]
  write.table(random.coef,file=paste0(initial_Occ_modelling_output,'Occ_random_effects_coeffs.csv'), quote = FALSE,append    = i > 1, 
              sep= ",", row.names = FALSE,col.names=FALSE)
}

for(i in random_demos){
  random.ef_temp <- as.data.frame(se.ranef(occ_final_model)[i])
  options(warn=-1);
  write.table(random.ef_temp,file=paste0(initial_Occ_modelling_output,'Occ_SE_random_demographics_effects.csv'), quote = FALSE,append    = i > 1, 
                               sep= ",", row.names = FALSE);options(warn=0)
}

for(i in occ_random_campaigns){
  random.ef_temp <- as.data.frame(se.ranef(occ_final_model)[i])
  colnames(random.ef_temp)[2] <- paste0(i,' S.E.')
  options(warn=-1);write.table(random.ef_temp[2],file=paste0(initial_Occ_modelling_output,'Occ_SE_random_campaigns_effects.csv'), quote = FALSE,append    = i > 1, 
                               sep= ",", row.names = FALSE);options(warn=0)
}

random_groupe <- c(random_demos,occ_random_campaigns)

if(length(random_demos)>0){
  random_group_demos <- data.frame(random_demos)
  names.random_group_demos <- c("class")
  setnames(random_group_demos,names.random_group_demos)
  random_group_demos$exposed <- 0
} else {
  random_group_demos <- c()
}

random_group_campaigns <- data.frame(occ_random_campaigns)
names.random_group_campaigns <- c("class")
setnames(random_group_campaigns,names.random_group_campaigns)
random_group_campaigns$exposed <- 1

random_group_exposed <- rbind( random_group_campaigns,random_group_demos)

random_group_data <- data.frame()
for(i in random_groupe){
  random.ef_temp <- as.data.frame(ranef(occ_final_model)[i])
  random.coef_temp <- as.data.table(random.ef_temp,keep.rownames=TRUE)
  random.coef <- random.coef_temp[,class:=i]
  names.random.coef <- c("level","group0","group1","class")
  setnames(random.coef ,names.random.coef)
  random.coef$row <- row.names(random.coef)
  random_group_data <-  rbind( random.coef,random_group_data)
}

random_group_SEdata <- data.frame()
for(i in random_groupe){
  random.sef_temp <- as.data.frame(se.ranef(occ_final_model)[i])
  random.secoef_temp <- as.data.table(random.sef_temp,keep.rownames=TRUE)
  random.secoef <- random.secoef_temp[,class:=i]
  names.random.secoef <- c("levelSE","SE_group0","SE_group1","class")
  setnames(random.secoef ,names.random.secoef)
  random.secoef$row <- row.names(random.secoef)
  random_group_SEdata <-  rbind( random.secoef,random_group_SEdata)
}

final_random <- merge(random_group_data,random_group_SEdata, by=c("class","row"))
final_random <- merge(final_random,random_group_exposed, by=c("class"))
View(final_random)
write.csv(final_random,file=paste0(initial_Occ_modelling_output,'Occ_random_effects.csv'), row.names = FALSE)

# create a txt file with the summary of the glmer model 
logsavefname <- "Occasion_Initial_Results.txt"
sink(paste0(initial_Occ_modelling_output,logsavefname))
print(Sys.time()-occ.start.time)
print(summary(occ_final_model))
print(ranef(occ_final_model))
sink()

for(i in c(occ_random_campaigns,random_demos)){
  pdf(paste0(initial_Occ_modelling_output,"Occ_Caterpillar_Plots ",i,".pdf"),width=7,height=5)
  print(ggCaterpillar(ranef(occ_final_model, condVar=TRUE)[i]))
  dev.off()
}

# save Occasion final dataset and final model
Occ_Random_Formula_final <- Formula_final
rm(finaldata_occ_num,temp_cor,cordata)

# save data in .Rdata file
save(finaldata_occ,Occ_Random_Formula_final,occ_final_model,occ_outliers_panid_to_remove,file=paste0(output,"saved_data/","Occasion_initial_model_data.Rdata"))

Occasion.Execution.Time <- Sys.time()-occ.start.time
cat(paste0("Occasion.Execution.Time: ",Occasion.Execution.Time))

#######################################
#-------------PENETRATION-------------#
#######################################

pen.start.time <- Sys.time()

#set response variable
y_var <- "Buyer_Pos_P1"

# drop variables with only one level
finaldata_pen <- as.data.table(initial_data)
indices <- sapply(finaldata_pen,function(x) length(unique(x))==1)
finaldata_pen <- finaldata_pen[,!indices,with=F]

#finaldata_pen <- finaldata_pen[sample(nrow(finaldata_pen),0.50*nrow(finaldata_pen)),]

# set mandatory variables
pen_mandatory_covariate <- c("Buyer_Pre_P1")

# final data for the modeling process
finaldata_pen <- as.data.frame(finaldata_pen)
finaldata_pen <- finaldata_pen[, !(names(finaldata_pen) %in% c("Trps_POS_P1","Dol_per_Trip_POS_P1","Trps_PRE_P1","Dol_per_Trip_PRE_P1","Nonbuyer_Pre_P1"))]
finaldata_pen <- finaldata_pen[!finaldata_pen$panid %in% occ_dolocc_outliers_panid_to_remove,]

# set initial model formula
Formula_fixed <- paste0(paste0(y_var,"~ ",paste0(names(finaldata_pen[,!names(finaldata_pen) %in% c(y_var,"panid",random_demos,random_campaigns,"Buyer_Pre_P1",scoring_vars)]),collapse=" + ")," "),"+ offset(log(Buyer_Pre_P1+1))")
options(warn=-1)

# run glm model
model_glm_total <- glm(Formula_fixed,data=finaldata_pen,family=binomial(link="logit"))
summary(model_glm_total)

##### Variable selection - Condition 1 #####
# Remove pvalues greater than 0.7 and rerun the glm model
pval_excluded_vars <- setdiff(c(rownames(as.data.frame(which(summary(model_glm_total)$coefficient[,4]>0.7))),setdiff(names(model_glm_total$coefficients),rownames(summary(model_glm_total)$coefficients))),pen_mandatory_covariate)
finaldata_pen <- finaldata_pen[, !names(finaldata_pen) %in% c(pval_excluded_vars)] 

# re-run glm model
Formula_fixed <- paste0(paste0(y_var,"~ ",paste0(names(finaldata_pen[,!names(finaldata_pen)%in%c(y_var,"panid",random_demos,random_campaigns,"Buyer_Pre_P1",scoring_vars)]),collapse=" + ")," "),"+ offset(log(Buyer_Pre_P1+1))")
model_glm_condition1 <- glm(Formula_fixed,data=finaldata_pen,family=binomial(link="logit"))
summary(model_glm_condition1)
rm(model_glm_total)

##### Variable selection - Condition 2 #####
# Remove variables that have z value lower than 2 and VIF greater than 15

# get variables with z value lower than 2
z_excluded_vars <- rownames(as.data.frame(which(abs(summary(model_glm_condition1)$coefficient[,3])<1.96)))

# get the variables with VIF greater than 15
multicol <- alias(model_glm_condition1)
multicols_toremove <- rownames(as.data.frame(multicol$Complete))

if (length(multicols_toremove)>0){
  finaldata_pen <- finaldata_pen[ , !names(finaldata_pen) %in% multicols_toremove] 
  Formula_fixed <- paste0(paste0(y_var,"~ ",paste0(names(finaldata_pen[,!names(finaldata_pen)%in%c(y_var,"panid",random_demos,random_campaigns,"Buyer_Pre_P1")]),collapse=" + ")," "),"+ offset(log(Buyer_Pre_P1+1))")
  model_glm_new <- glm(Formula_fixed,data=finaldata_pen,family=binomial(link="logit"))  
} else {
  model_glm_new <- model_glm_condition1
}

vif_output <- as.data.frame(vif(model_glm_new))
vif_to_remove <- rownames(vif_output)[(vif(model_glm_new)) > 15]

# final vars to exclude under condition 2
vars_condition2 <- setdiff(intersect(z_excluded_vars,vif_to_remove),pen_mandatory_covariate)

# re-run glm model
finaldata_pen <- finaldata_pen[ , !names(finaldata_pen) %in% c(vars_condition2,pval_excluded_vars)] 
Formula_fixed <- paste0(paste0(y_var,"~ ",paste0(names(finaldata_pen[,!names(finaldata_pen)%in%c(y_var,"panid",random_demos,random_campaigns,"Buyer_Pre_P1",scoring_vars)]),collapse=" + ")," ")," + offset(log(Buyer_Pre_P1+1))")
model_glm <- glm(Formula_fixed,data=finaldata_pen,family=binomial(link="logit"))
summary(model_glm)
rm(model_glm_condition1)

# p-values and signs checks
pvalues <- summary(model_glm)$coefficients[,4][-1]
keep_covariate <- names(pvalues[pvalues<pvalue_lvl])

# final cleaning of first model based on signs and pvalues
neutralvars <- setdiff(names(finaldata_pen),c(positivevars,negativevars))
coeffs <- sign(summary(model_glm)$coefficients[,1])
varstokeep <- c();initialvars <- c()
for (i in negativevars) if (i %in% names(coeffs[coeffs<0])) varstokeep <- c(varstokeep,i)
for (j in positivevars) if (j %in% names(coeffs[coeffs>0])) varstokeep <- c(varstokeep,j)
for (l in neutralvars) varstokeep=c(varstokeep,l)
for (k in varstokeep) if (k %in% keep_covariate) initialvars <- c(initialvars,k)

#set the formula for the second model after cleaning the first model
Formula_final <- paste0(paste0(y_var,"~ ",paste0(initialvars,collapse=" + ")," "),"+ group + offset(log(Buyer_Pre_P1+1))")
model_glm_2 <- glm(Formula_final,data=finaldata_pen,family=binomial(link="logit"))
summary(model_glm_2)
rm(model_glm)

# cleaning of second model based on signs and pvalues
pvalues_2 <- summary(model_glm_2)$coefficients[,4][-1]
keep_covariate_2 <- names(pvalues_2[pvalues_2<pvalue_lvl])
coeffs_2 <- sign(summary(model_glm_2)$coefficients[,1])
varstokeep_2 <- c();vars_2 <- c()
for (i in negativevars) if (i %in% names(coeffs_2[coeffs_2<0])) varstokeep_2 <- c(varstokeep_2,i)
for (j in positivevars) if (j %in% names(coeffs_2[coeffs_2>0])) varstokeep_2 <- c(varstokeep_2,j)
for (l in neutralvars) varstokeep_2 <- c(varstokeep_2,l)
for (k in varstokeep_2) if (k %in% keep_covariate_2) vars_2 <- c(vars_2,k)

#######################################
#------------ Final Model ------------#
#######################################

#set the formula for the final model
Formula_final <- paste0(paste0(y_var,"~ ",paste0(vars_2,collapse=" + ")," "),"+ group + offset(log(Buyer_Pre_P1+1))")
pen_final_model <- glm(Formula_final,data=finaldata_pen,family=binomial(link="logit"))
summary(pen_final_model)
rm(model_glm_2)

# final model
# final cleaning of model based on signs
pvalues_2 <- summary(pen_final_model)$coefficients[,4][-1]
keep_covariate_2 <- names(pvalues_2[pvalues_2<pvalue_lvl])
coeffs <- sign(summary(pen_final_model)$coefficients[,1])
finalvars1 <- c();finalvars <- c()
for (i in negativevars) if (i %in% names(coeffs[coeffs<0])) finalvars1 <- c(finalvars1,i)
for (j in positivevars) if (j %in% names(coeffs[coeffs>0])) finalvars1 <- c(finalvars1,j)
for (l in neutralvars) if (l %in% keep_covariate_2) finalvars1 <- c(finalvars1,l)
for (k in finalvars1) if (k %in% keep_covariate_2) finalvars <- c(finalvars,k)

# final correlation check (find pair vars with correlation greater then 0.8 and drop the one with bigger pvalue)
finaldata_pen_num <- as.data.table(finaldata_pen[,names(finaldata_pen) %in% finalvars])
t1 <- as.data.frame(finaldata_pen_num[,sapply(finaldata_pen_num,is.numeric)])
finaldata_pen_num <- as.data.frame(finaldata_pen_num)
finaldata_pen_num <- finaldata_pen_num[,c(which(t1==T))]
temp_cor <- as.matrix(cor(finaldata_pen_num))
temp_cor[lower.tri(temp_cor, diag = T)] <- NA
temp_corIndex <- which(temp_cor > 0.8 |temp_cor < (-0.8), arr.ind = TRUE)
cordata <- data.table(col1=colnames(temp_cor)[temp_corIndex[,1]],col2=colnames(temp_cor)[temp_corIndex[,2]])

if(nrow(cordata)>0) {
  drop_cor_vars <- c()
  for (i in 1:nrow(cordata)) {
    if(cordata[i][,1,with=F]=="group"|cordata[i][,2,with=F]=="group"){
      if(cordata[i][,1,with=F]=="group") drop_cor_vars <- c(drop_cor_vars,as.character(cordata[i][,2,with=F]))
      if(cordata[i][,2,with=F]=="group") drop_cor_vars <- c(drop_cor_vars,as.character(cordata[i][,1,with=F]))
    } else {
      if (summary(pen_final_model)$coefficients[which(rownames(summary(pen_final_model)$coefficients)==as.character(cordata[i][,1,with=F])),4] > summary(pen_final_model)$coefficients[which(rownames(summary(pen_final_model)$coefficients)==as.character(cordata[i][,2,with=F])),4]) {
        drop_cor_vars <- c(drop_cor_vars,as.character(cordata[i][,1,with=F]))
      } else {
        drop_cor_vars <- c(drop_cor_vars,as.character(cordata[i][,2,with=F]))
      }
    }
  }
  finalvars <- setdiff(finalvars,drop_cor_vars)
}

#set the final formula
Formula_final <- paste0(paste0(y_var,"~ ",paste0(finalvars,collapse=" + ")," ")," + group  + offset(log(Buyer_Pre_P1+1))")
pen_final_model <- glm(Formula_final,data=finaldata_pen,family=binomial(link="logit"))
summary(pen_final_model)

options(warn=0)

# # kruskal wallis test for pair comparison of publisher's levels only for penetration
# # Pen model
# finaldata_pen <- as.data.table(finaldata_pen)
# finaldata_pen[,Prediction:=predict(pen_final_model, type="response")]
# finaldata_pen <- droplevels(finaldata_pen)
# 
# kruskal_var <- 'Prediction'
# kruskal_randoms <- c()
# for(i in random_campaigns){
#   kt <- kruskal.test(eval(parse(text=kruskal_var))~eval(parse(text=i)),data=finaldata_pen)
#   print(i);print(kt)
#   if(kt$p.value<0.10){
#     kruskal_randoms <- append(kruskal_randoms,i)
#   }
# }
#
# warning('In case of aggregation based on Kruskal-Wallis test, make sure that the other level of each sub-campaign is named as Other_XXX')
# for(i in kruskal_randoms){
#   lev <- setdiff(levels(finaldata_pen[,eval(parse(text=i))]),'none')
#   if(length(lev)>2){
#     aggr <- c()
#     for(j in 1:(length(lev)-1)){
#       for( k in (j+1):(length(lev))){       
#         df <- subset(finaldata_pen, eval(parse(text=i))  %in% c(lev[j],lev[k]))
#         l1 <- list(c(lev[j],lev[k]),P_Value=try(round(kruskal.test(df[,eval(parse(text=kruskal_var))]~df[,eval(parse(text=i))])$p.value,5),silent=TRUE))
#         if(l1[2]=="NaN") next
#         vectorElements <- unlist(l1)
#         listnames <- names(vectorElements) <- c(paste0(i,':'),paste0(i,':'),'P_value:')
#         listout <- paste(listnames,vectorElements)
#         write.table(listout,file=paste0(final_Occ_modelling_output,'Kruskal.csv'), row.names = FALSE, col.names = FALSE, quote = FALSE,append=TRUE)  
#         if(l1[['P_Value']]<0.05){
#           aggr <- append(aggr,c(sapply(l1[1],`[`,1),sapply(l1[1],`[`,2)))
#           aggr <- setdiff(aggr,c('none',paste0("Other_",i)))
#         }
#       }  
#     }
#     if(length(unique(aggr))>0){
#       finaldata_pen[!eval(parse(text=i)) %in% c(aggr,'none'),eval(parse(text=i)):=paste0("Other_",i)]
#     } else {
#       finaldata_pen[,eval(parse(text=i)):=NULL]
#     }
#   }
# }

finaldata_pen <- as.data.table(finaldata_pen)
finaldata_pen <- droplevels(finaldata_pen)
pen_random_campaigns <- intersect(random_campaigns,names(finaldata_pen))

if(length(pen_random_campaigns)>0){
  pen_random_campaigns_factors <- paste0("+(0+group|",pen_random_campaigns,")")
} else {
  pen_random_campaigns_factors <- NULL
}
random_demos_factors <- paste0( "+(0+group|",random_demos,")" )

##################################################################################################################

# run glmer model
pen.mixed.start.time <- Sys.time()
Formula_final <- paste0(c(paste0(paste0(y_var,"~ ",paste0(finalvars,collapse=" + ")," "),"+ offset(log(Buyer_Pre_P1+1)) + group"),pen_random_campaigns_factors),collapse='')
pen_final_model <- glmer(Formula_final,data=finaldata_pen,family=binomial(link="logit"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
pen.mixed.model.execution.time <- Sys.time()-pen.mixed.start.time
out_resid <- order(residuals(pen_final_model),decreasing = T)
finaldata_pen <- as.data.table(finaldata_pen)
pen_total_outliers_panid <- finaldata_pen[out_resid,panid]
segment1_panid <- data_NB_NE_B[,panid]
segment2_panid <- data_B_E_NB[,panid]
outliers1_panid <- intersect(pen_total_outliers_panid,segment1_panid)
outliers2_panid <- intersect(pen_total_outliers_panid,segment2_panid)
pen_outliers_panid_segm <- c(outliers1_panid,outliers2_panid)

# remove outliers if needed (negative estimation for exposure)
pen_outliers_panid_to_remove <- c()
pen_outliers_panid_segm_new <- c()
pen_outliers_panid_to_remove_threshold <- c()

flag <- c()
coef_flag <- c()
for(i in setdiff(levels(finaldata_pen$group),"0")) flag <- append(flag,paste0('group',i))

for(j in flag){
  coef_flag <- append(coef_flag,coef(summary(pen_final_model))[j,"Estimate"])
}

if(length(pen_random_campaigns)>0){
  min_random_coeff <- min(unlist(lapply(ranef(pen_final_model)[pen_random_campaigns],FUN=min)))
  max_random_coeff <- max(unlist(lapply(ranef(pen_final_model)[pen_random_campaigns],FUN=max)))
} else {
  min_random_coeff <- 0
}

if(min_random_coeff < -0.05){
  out_resid_positive <- order(residuals(pen_final_model),decreasing = T)
  out_resid_negative <- order(residuals(pen_final_model),decreasing = F)
  pen_outliers_panid_positive_extreme_panid <- finaldata_pen[out_resid_positive,panid]
  pen_outliers_panid_negative_extreme_panid <- finaldata_pen[out_resid_negative,panid]
  for(i in 1:length(pen_random_campaigns)){
    min_random_coeff <- min(unlist(lapply(ranef(pen_final_model)[i],FUN=min)))
    if(min_random_coeff < -0.05){
      max_random_coeff <- max(unlist(lapply(ranef(pen_final_model)[i],FUN=max)))
      random_frame <- as.data.frame(ranef(pen_final_model)[i])
      random_table <- as.data.table(random_frame)
      random_table[,level:=row.names(random_frame)]
      negative_extreme_levels <- random_table[eval(parse(text=paste0(names(ranef(pen_final_model)[i]),'.group1')))==min_random_coeff,]$level
      positive_extreme_levels <- random_table[eval(parse(text=paste0(names(ranef(pen_final_model)[i]),'.group1')))== max_random_coeff]$level
      negative_extreme_panid <- finaldata_pen[eval(parse(text=paste0(names(ranef(pen_final_model)[i])))) %in% negative_extreme_levels,panid] # -13%
      positive_extreme_panid  <- finaldata_pen[eval(parse(text=paste0(names(ranef(pen_final_model)[i])))) %in% positive_extreme_levels,panid] # +12%
      pen_outliers_panid_segm1 <- intersect(pen_outliers_panid_negative_extreme_panid,negative_extreme_panid);pen_outliers_panid_segm1 <- pen_outliers_panid_segm1[1:round(0.02*(length(negative_extreme_panid)))]
      if(max_random_coeff > 0.05) {
        pen_outliers_panid_segm2 <- intersect(pen_outliers_panid_positive_extreme_panid,positive_extreme_panid)
        pen_outliers_panid_segm2 <- pen_outliers_panid_segm2[1:round(0.02*(length(positive_extreme_panid)))]
      } else {
        pen_outliers_panid_segm2 <- c()
      }
      pen_outliers_panid_segm_new <- c(pen_outliers_panid_segm1,pen_outliers_panid_segm2)
      finaldata_pen <- finaldata_pen[!finaldata_pen$panid %in% pen_outliers_panid_segm_new,]
    }
  }
  pen_final_model <- glmer(Formula_final,data=finaldata_pen,family=binomial(link="logit"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
  min_random_coeff <- min(unlist(lapply(ranef(pen_final_model)[pen_random_campaigns],FUN=min)))
  if(min_random_coeff < -0.05){
    for(i in 1:length(pen_random_campaigns)){
      min_random_coeff <- min(unlist(lapply(ranef(pen_final_model)[i],FUN=min)))
      if(min_random_coeff < -0.05){
        max_random_coeff <- max(unlist(lapply(ranef(pen_final_model)[i],FUN=max)))
        random_frame <- as.data.frame(ranef(pen_final_model)[i])
        random_table <- as.data.table(random_frame)
        random_table[,level:=row.names(random_frame)]
        negative_extreme_levels <- random_table[eval(parse(text=paste0(names(ranef(pen_final_model)[i]),'.group1')))==min_random_coeff,]$level
        positive_extreme_levels <- random_table[eval(parse(text=paste0(names(ranef(pen_final_model)[i]),'.group1')))== max_random_coeff]$level
        negative_extreme_panid <- finaldata_pen[eval(parse(text=paste0(names(ranef(pen_final_model)[i])))) %in% negative_extreme_levels,panid] # -13%
        positive_extreme_panid  <- finaldata_pen[eval(parse(text=paste0(names(ranef(pen_final_model)[i])))) %in% positive_extreme_levels,panid] # +12%
        pen_outliers_panid_segm1 <- intersect(pen_outliers_panid_negative_extreme_panid,negative_extreme_panid);pen_outliers_panid_segm1 <- pen_outliers_panid_segm1[1:round(0.05*(length(negative_extreme_panid)))]
        if(max_random_coeff > 0.05) {
          pen_outliers_panid_segm2 <- intersect(pen_outliers_panid_positive_extreme_panid,positive_extreme_panid)
          pen_outliers_panid_segm2 <- pen_outliers_panid_segm2[1:round(0.05*(length(positive_extreme_panid)))]
        } else {
          pen_outliers_panid_segm2 <- c()
        }
        pen_outliers_panid_segm_new <- c(pen_outliers_panid_segm1,pen_outliers_panid_segm2)
        finaldata_pen <- finaldata_pen[!finaldata_pen$panid %in% pen_outliers_panid_segm_new,]
      }
    }
    pen_final_model <- glmer(Formula_final,data=finaldata_pen,family=binomial(link="logit"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
    min_random_coeff <- min(unlist(lapply(ranef(pen_final_model)[pen_random_campaigns],FUN=min)))
  }
  if(min_random_coeff < -0.05){
    for(i in 1:length(pen_random_campaigns)){
      min_random_coeff <- min(unlist(lapply(ranef(pen_final_model)[i],FUN=min)))
      if(min_random_coeff < -0.05){
        max_random_coeff <- max(unlist(lapply(ranef(pen_final_model)[i],FUN=max)))
        random_frame <- as.data.frame(ranef(pen_final_model)[i])
        random_table <- as.data.table(random_frame)
        random_table[,level:=row.names(random_frame)]
        negative_extreme_levels <- random_table[eval(parse(text=paste0(names(ranef(pen_final_model)[i]),'.group1')))==min_random_coeff,]$level
        positive_extreme_levels <- random_table[eval(parse(text=paste0(names(ranef(pen_final_model)[i]),'.group1')))== max_random_coeff]$level
        negative_extreme_panid <- finaldata_pen[eval(parse(text=paste0(names(ranef(pen_final_model)[i])))) %in% negative_extreme_levels,panid] # -13%
        positive_extreme_panid  <- finaldata_pen[eval(parse(text=paste0(names(ranef(pen_final_model)[i])))) %in% positive_extreme_levels,panid] # +12%
        pen_outliers_panid_segm1 <- intersect(pen_outliers_panid_negative_extreme_panid,negative_extreme_panid);pen_outliers_panid_segm1 <- pen_outliers_panid_segm1[1:round(0.10*(length(negative_extreme_panid)))]
        if(max_random_coeff > 0.05) {
          pen_outliers_panid_segm2 <- intersect(pen_outliers_panid_positive_extreme_panid,positive_extreme_panid)
          pen_outliers_panid_segm2 <- pen_outliers_panid_segm2[1:round(0.10*(length(positive_extreme_panid)))]
        } else {
          pen_outliers_panid_segm2 <- c()
        }
        pen_outliers_panid_segm_new <- c(pen_outliers_panid_segm1,pen_outliers_panid_segm2)
        finaldata_pen <- finaldata_pen[!finaldata_pen$panid %in% pen_outliers_panid_segm_new,]
      }
    }
    pen_final_model <- glmer(Formula_final,data=finaldata_pen,family=binomial(link="logit"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
    min_random_coeff <- min(unlist(lapply(ranef(pen_final_model)[pen_random_campaigns],FUN=min)))
  }
}

coef_flag <- c()
pvalue_flag <- c()
for(j in flag){
  coef_flag <- append(coef_flag,coef(summary(pen_final_model))[j,"Estimate"])
  pvalue_flag <- append(pvalue_flag,coef(summary(pen_final_model))[j,"Pr(>|z|)"])
}

if(length(pen_random_campaigns)>0){
  min_random_coeff <- min(unlist(lapply(ranef(pen_final_model)[pen_random_campaigns],FUN=min)))
  max_random_coeff <- max(unlist(lapply(ranef(pen_final_model)[pen_random_campaigns],FUN=max)))
} else {
  min_random_coeff <- 0
}

# drop outliers in case of high exposure estimation (in case we want a lower group1 coef to meet business requirements)
# segment_panid <- pen_reduction[,panid]
# outliers_panid <- intersect(pen_total_outliers_panid,segment_panid)
# pen_outliers_panid_to_remove_threshold <- outliers_panid[c(1:round(0.10*(length(outliers_panid))))]
# finaldata_pen <- finaldata_pen[!finaldata_pen$panid %in% pen_outliers_panid_to_remove_threshold,]
# pen_final_model <- glmer(Formula_final,data=finaldata_pen,family=binomial(link="logit"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))

if(min(coef_flag) + min_random_coeff < 0 | max(pvalue_flag)>0.20){
  cat('\n')
  cat("removing outliers ...\nEstimated Iteration Execution ");print(pen.mixed.model.execution.time)
  cat("\nIteration 1 ... Start Time: ");print(Sys.time())
  pen_outliers_panid_to_remove <- pen_outliers_panid_segm[c(1:round(0.05*(nrow(data_NB_NE_B)+nrow(data_B_E_NB))))]
  finaldata_pen <- finaldata_pen[!finaldata_pen$panid %in% pen_outliers_panid_to_remove,]
  pen_final_model <- glmer(Formula_final,data=finaldata_pen,family=binomial(link="logit"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
  coef_flag <- c()
  pvalue_flag <- c()
  for(j in flag){
    coef_flag <- append(coef_flag,coef(summary(pen_final_model))[j,"Estimate"])
    pvalue_flag <- append(pvalue_flag,coef(summary(pen_final_model))[j,"Pr(>|z|)"])
  }
  min_random_coeff <- min(unlist(lapply(ranef(pen_final_model)[pen_random_campaigns],FUN=min)))
  if(min(coef_flag) + min_random_coeff < 0 | max(pvalue_flag)>0.20){
    pen_outliers_panid_to_remove <- pen_outliers_panid_segm[c(1:round(0.10*(nrow(data_NB_NE_B)+nrow(data_B_E_NB))))]  
    finaldata_pen <- finaldata_pen[!finaldata_pen$panid %in% pen_outliers_panid_to_remove,]
    cat("\nIteration 2 ... Start Time: ");print(Sys.time())
    pen_final_model <- glmer(Formula_final,data=finaldata_pen,family=binomial(link="logit"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
    coef_flag <- c()
    pvalue_flag <- c()
    for(j in flag){
      coef_flag <- append(coef_flag,coef(summary(pen_final_model))[j,"Estimate"])
      pvalue_flag <- append(pvalue_flag,coef(summary(pen_final_model))[j,"Pr(>|z|)"])
    }
    min_random_coeff <- min(unlist(lapply(ranef(pen_final_model)[pen_random_campaigns],FUN=min)))
    if(min(coef_flag) + min_random_coeff < 0 | max(pvalue_flag)>0.20){
      pen_outliers_panid_to_remove <- pen_outliers_panid_segm[c(1:round(0.15*(nrow(data_NB_NE_B)+nrow(data_B_E_NB))))]  
      finaldata_pen <- finaldata_pen[!finaldata_pen$panid %in% pen_outliers_panid_to_remove,]
      cat("\nIteration 3 ... Start Time: ");print(Sys.time())
      pen_final_model <- glmer(Formula_final,data=finaldata_pen,family=binomial(link="logit"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
      coef_flag <- c()
      pvalue_flag <- c()
      for(j in flag){
        coef_flag <- append(coef_flag,coef(summary(pen_final_model))[j,"Estimate"])
        pvalue_flag <- append(pvalue_flag,coef(summary(pen_final_model))[j,"Pr(>|z|)"])
      }
      min_random_coeff <- min(unlist(lapply(ranef(pen_final_model)[pen_random_campaigns],FUN=min)))
      if(min(coef_flag) + min_random_coeff < 0 | max(pvalue_flag)>0.20){
        pen_outliers_panid_to_remove <- pen_outliers_panid_segm[c(1:round(0.20*(nrow(data_NB_NE_B)+nrow(data_B_E_NB))))]  
        finaldata_pen <- finaldata_pen[!finaldata_pen$panid %in% pen_outliers_panid_to_remove,]
        cat("\nFinal Iteration ... Start Time: ");print(Sys.time())
        pen_final_model <- glmer(Formula_final,data=finaldata_pen,family=binomial(link="logit"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
      }
    }
  }
}

write.csv(as.data.frame(summary(pen_final_model)$coefficients),file=paste0(final_Pen_modelling_output,"Pen_fixed_effects.csv"))
write.csv(as.data.frame(summary(pen_final_model)$coefficients),file=paste0(scoring_Pen_output,"Pen_fixed_effects.csv"))

# create output for random effects
randoms <- as.data.frame(augment.ranef.mer(ranef(pen_final_model,condVar=TRUE)))
random.ef_temp.report <- randoms[,!names(randoms) %in% c("qq")]
write.csv(random.ef_temp.report,file=paste0(final_Pen_modelling_output,"Pen_random_effects_pvalue.csv"),row.names=FALSE)

for(i in pen_random_campaigns){
  random.ef_temp <- as.data.frame(ranef(pen_final_model)[i])
  random.coef_temp <- as.data.table(random.ef_temp[2],keep.rownames=TRUE)
  random.coef <- random.coef_temp[,label:=names(random.coef_temp)[2]]
  write.table(random.coef,file=paste0(final_Pen_modelling_output,'Pen_random_effects_coeffs.csv'), quote = FALSE,append    = i > 1, 
              sep= ",", row.names = FALSE,col.names=FALSE)
}

for(i in pen_random_campaigns){
  random.ef_temp <- as.data.frame(se.ranef(pen_final_model)[i])
  colnames(random.ef_temp)[2] <- paste0(i,' S.E.')
  options(warn=-1);write.table(random.ef_temp[2],file=paste0(final_Pen_modelling_output,'Pen_SE_random_effects.csv'), quote = FALSE,append    = i > 1, 
                               sep= ",", row.names = FALSE);options(warn=0)
}

random_demos <- c()
random_groupe <- c(random_demos,pen_random_campaigns)

if(length(random_demos)>0){
  random_group_demos <- data.frame(random_demos)
  names.random_group_demos <- c("class")
  setnames(random_group_demos,names.random_group_demos)
  random_group_demos$exposed <- 0
} else {
  random_group_demos <- c()
}

random_group_campaigns <- data.frame(pen_random_campaigns)
names.random_group_campaigns <- c("class")
setnames(random_group_campaigns,names.random_group_campaigns)
random_group_campaigns$exposed <- 1

random_group_exposed <- rbind( random_group_campaigns,random_group_demos)

random_group_data <- data.frame()
for(i in random_groupe){
#for(i in allrandoms){  
  random.ef_temp <- as.data.frame(ranef(pen_final_model)[i])
  random.coef_temp <- as.data.table(random.ef_temp,keep.rownames=TRUE)
  random.coef <- random.coef_temp[,class:=i]
  names.random.coef <- c("level","group0","group1","class")
  setnames(random.coef ,names.random.coef)
  random.coef$row <- row.names(random.coef)
  random_group_data <-  rbind( random.coef,random_group_data)
}

random_group_SEdata <- data.frame()
for(i in random_groupe){
  random.sef_temp <- as.data.frame(se.ranef(pen_final_model)[i])
  random.secoef_temp <- as.data.table(random.sef_temp,keep.rownames=TRUE)
  random.secoef <- random.secoef_temp[,class:=i]
  names.random.secoef <- c("levelSE","SE_group0","SE_group1","class")
  setnames(random.secoef ,names.random.secoef)
  random.secoef$row <- row.names(random.secoef)
  random_group_SEdata <-  rbind( random.secoef,random_group_SEdata)
}

final_random <- merge(random_group_data,random_group_SEdata, by=c("class","row"))
final_random <- merge(final_random,random_group_exposed, by=c("class"))
View(final_random)
write.csv(final_random,file=paste0(scoring_Pen_output,'Pen_random_effects.csv'), row.names = FALSE)

# create a txt file with the summary of the glmer model 
logsavefname <- "Pen_Final_Results.txt"
sink(paste0(final_Pen_modelling_output,logsavefname))
print(Sys.time()-pen.start.time)
print(summary(pen_final_model))
print(ranef(pen_final_model))
sink()

for(i in pen_random_campaigns){
  pdf(paste0(final_Pen_modelling_output,"Pen_Caterpillar_Plots ",i,".pdf"),width=7,height=5)
  print(ggCaterpillar(ranef(pen_final_model, condVar=TRUE)[i]))
  dev.off()
}

# save Penetration final dataset and final model
Pen_Formula_final <- Formula_final
pen_outliers_panid_to_remove <- c(pen_outliers_panid_to_remove,pen_outliers_panid_segm_new,pen_outliers_panid_to_remove_threshold)
rm(finaldata_pen_num,temp_cor,cordata)

# save data in .Rdata file
save(finaldata_pen,Pen_Formula_final,pen_final_model,pen_outliers_panid_to_remove,file=paste0(output,"saved_data/","pen_final_model_data.Rdata"))

# Check and Rerun Occasion and DolOcc models due to outliers removal if needed
if(length(pen_outliers_panid_to_remove)>0){
  #load("Occasion_initial_model_data.Rdata")
  cat('\n')
  cat("Re-Running Occasion Model ...")
  finaldata_occ <- finaldata_occ[!finaldata_occ$panid %in% pen_outliers_panid_to_remove,]
  
  # run glmer model
  occ_final_model <- glmer(Occ_Random_Formula_final,data=finaldata_occ,family=poisson(link="log"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
  
}

# save data in .Rdata file
save(finaldata_occ,occ_final_model,Occ_Random_Formula_final,file=paste0(output,"saved_data/","Occasion_final_model_data.Rdata"))

write.csv(as.data.frame(summary(occ_final_model)$coefficients),file=paste0(final_Occ_modelling_output,"Occ_fixed_effects.csv"))
write.csv(as.data.frame(summary(occ_final_model)$coefficients),file=paste0(scoring_Occ_output,"Occ_fixed_effects.csv"))

# create output for random effects
randoms <- as.data.frame(augment.ranef.mer(ranef(occ_final_model,condVar=TRUE)))
random.ef_temp.report <- randoms[,!names(randoms) %in% c("qq")]
write.csv(random.ef_temp.report,file=paste0(final_Occ_modelling_output,"Occ_random_effects_pvalue.csv"),row.names=FALSE)

for(i in random_demos){
  random.ef_temp <- as.data.frame(ranef(occ_final_model)[i])
  random.coef_temp <- as.data.table(random.ef_temp,keep.rownames=TRUE)
  random.coef <- random.coef_temp[,class:=i]
  names(random.coef) <- c("","group0","group1","class")
  options(warn=-1);write.table(random.coef,file=paste0(final_Occ_modelling_output,'Occ_random_demographics_effects.csv'), quote = FALSE,append    = i > 1, 
                               sep= ",", row.names = FALSE,col.names=ifelse(i %in% random_demos[1], TRUE, FALSE));options(warn=0)
}

for(i in occ_random_campaigns){
  random.ef_temp <- as.data.frame(ranef(occ_final_model)[i])
  random.coef_temp <- as.data.table(random.ef_temp[2],keep.rownames=TRUE)
  random.coef <- random.coef_temp[,label:=names(random.coef_temp)[2]]
  write.table(random.coef,file=paste0(final_Occ_modelling_output,'Occ_random_effects_coeffs.csv'), quote = FALSE,append    = i > 1, 
              sep= ",", row.names = FALSE,col.names=FALSE)
}

for(i in random_demos){
  random.ef_temp <- as.data.frame(se.ranef(occ_final_model)[i])
  options(warn=-1);write.table(random.ef_temp,file=paste0(final_Occ_modelling_output,'Occ_SE_random_demographics_effects.csv'), quote = FALSE,append    = i > 1, 
                               sep= ",", row.names = FALSE);options(warn=0)
}

for(i in occ_random_campaigns){
  random.ef_temp <- as.data.frame(se.ranef(occ_final_model)[i])
  colnames(random.ef_temp)[2] <- paste0(i,' S.E.')
  options(warn=-1);write.table(random.ef_temp[2],file=paste0(final_Occ_modelling_output,'Occ_SE_random_campaigns_effects.csv'), quote = FALSE,append    = i > 1, 
                               sep= ",", row.names = FALSE);options(warn=0)
}

random_groupe <- c(random_demos,occ_random_campaigns)

if(length(random_demos)>0){
  random_group_demos <- data.frame(random_demos)
  names.random_group_demos <- c("class")
  setnames(random_group_demos,names.random_group_demos)
  random_group_demos$exposed <- 0
} else {
  random_group_demos <- c()
}

random_group_campaigns <- data.frame(occ_random_campaigns)
names.random_group_campaigns <- c("class")
setnames(random_group_campaigns,names.random_group_campaigns)
random_group_campaigns$exposed <- 1

random_group_exposed <- rbind( random_group_campaigns,random_group_demos)

random_group_data <- data.frame()
for(i in random_groupe){
  random.ef_temp <- as.data.frame(ranef(occ_final_model)[i])
  random.coef_temp <- as.data.table(random.ef_temp,keep.rownames=TRUE)
  random.coef <- random.coef_temp[,class:=i]
  names.random.coef <- c("level","group0","group1","class")
  setnames(random.coef ,names.random.coef)
  random.coef$row <- row.names(random.coef)
  random_group_data <-  rbind( random.coef,random_group_data)
}

random_group_SEdata <- data.frame()
for(i in random_groupe){
  random.sef_temp <- as.data.frame(se.ranef(occ_final_model)[i])
  random.secoef_temp <- as.data.table(random.sef_temp,keep.rownames=TRUE)
  random.secoef <- random.secoef_temp[,class:=i]
  names.random.secoef <- c("levelSE","SE_group0","SE_group1","class")
  setnames(random.secoef ,names.random.secoef)
  random.secoef$row <- row.names(random.secoef)
  random_group_SEdata <-  rbind( random.secoef,random_group_SEdata)
}

final_random <- merge(random_group_data,random_group_SEdata, by=c("class","row"))
final_random <- merge(final_random,random_group_exposed, by=c("class"))
View(final_random)
write.csv(final_random,file=paste0(scoring_Occ_output,'Occ_random_effects.csv'), row.names = FALSE)

# create a txt file with the summary of the glmer model 
logsavefname <- "Occasion_Final_Results.txt"
sink(paste0(final_Occ_modelling_output,logsavefname))
print(Sys.time()-occ.start.time)
print(summary(occ_final_model))
print(ranef(occ_final_model))
sink()

for(i in c(occ_random_campaigns,random_demos)){
  pdf(paste0(final_Occ_modelling_output,"Occ_Caterpillar_Plots ",i,".pdf"),width=7,height=5)
  print(ggCaterpillar(ranef(occ_final_model, condVar=TRUE)[i]))
  dev.off()
}

if(length(unique(c(occ_outliers_panid_to_remove,pen_outliers_panid_to_remove)))>0){
  #load("DolOcc_initial_model_data.Rdata")
  cat('\n')
  cat("Re-Running DolOcc Model ...")
  finaldata_dolocc <- finaldata_dolocc[!finaldata_dolocc$panid %in% unique(c(occ_outliers_panid_to_remove,pen_outliers_panid_to_remove)),]
  
  # run glmer model
  dolocc_final_model <- glmer(dolocc_Formula_final,data=finaldata_dolocc,family=Gamma(link="log"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))
}

# save data in .Rdata file
save(finaldata_dolocc,dolocc_final_model,dolocc_Formula_final,final_DollOcc_modelling_output,file=paste0(output,"saved_data/","DolOcc_final_model_data.Rdata"))

write.csv(as.data.frame(summary(dolocc_final_model)$coefficients),file=paste0(final_DollOcc_modelling_output,"DolOcc_fixed_effects.csv"))
write.csv(as.data.frame(summary(dolocc_final_model)$coefficients),file=paste0(scoring_DolOcc_output,"DolOcc_fixed_effects.csv"))

# create output for random effects
randoms <- as.data.frame(augment.ranef.mer(ranef(dolocc_final_model,condVar=TRUE)))
random.ef_temp.report <- randoms[,!names(randoms) %in% c("qq")]
write.csv(random.ef_temp.report,file=paste0(final_DollOcc_modelling_output,"DolOcc_random_effects_pvalue.csv"),row.names=FALSE)

for(i in dolocc_random_campaigns){
  random.ef_temp <- as.data.frame(ranef(dolocc_final_model)[i])
  random.coef_temp <- as.data.table(random.ef_temp[2],keep.rownames=TRUE)
  random.coef <- random.coef_temp[,label:=names(random.coef_temp)[2]]
  write.table(random.coef,file=paste0(final_DollOcc_modelling_output,'DolOcc_random_effects_coeffs.csv'), quote = FALSE,append    = i > 1, 
              sep= ",", row.names = FALSE,col.names=FALSE)
}

for(i in dolocc_random_campaigns){
  random.ef_temp <- as.data.frame(se.ranef(dolocc_final_model)[i])
  colnames(random.ef_temp)[2] <- paste0(i,' S.E.')
  options(warn=-1);write.table(random.ef_temp[2],file=paste0(final_DollOcc_modelling_output,'DolOcc_SE_random_effects.csv'), quote = FALSE,append    = i > 1, 
                               sep= ",", row.names = FALSE);options(warn=0)
}

random_demos <- c()
random_groupe <- c(random_demos,dolocc_random_campaigns)

if(length(random_demos)>0){
  random_group_demos <- data.frame(random_demos)
  names.random_group_demos <- c("class")
  setnames(random_group_demos,names.random_group_demos)
  random_group_demos$exposed <- 0
} else {
  random_group_demos <- c()
}

random_group_campaigns <- data.frame(dolocc_random_campaigns)
names.random_group_campaigns <- c("class")
setnames(random_group_campaigns,names.random_group_campaigns)
random_group_campaigns$exposed <- 1

random_group_exposed <- rbind( random_group_campaigns,random_group_demos)

random_group_data <- data.frame()
for(i in random_groupe){
  random.ef_temp <- as.data.frame(ranef(dolocc_final_model)[i])
  random.coef_temp <- as.data.table(random.ef_temp,keep.rownames=TRUE)
  random.coef <- random.coef_temp[,class:=i]
  names.random.coef <- c("level","group0","group1","class")
  setnames(random.coef ,names.random.coef)
  random.coef$row <- row.names(random.coef)
  random_group_data <-  rbind( random.coef,random_group_data)
}

random_group_SEdata <- data.frame()
for(i in random_groupe){
  random.sef_temp <- as.data.frame(se.ranef(dolocc_final_model)[i])
  random.secoef_temp <- as.data.table(random.sef_temp,keep.rownames=TRUE)
  random.secoef <- random.secoef_temp[,class:=i]
  names.random.secoef <- c("levelSE","SE_group0","SE_group1","class")
  setnames(random.secoef ,names.random.secoef)
  random.secoef$row <- row.names(random.secoef)
  random_group_SEdata <-  rbind( random.secoef,random_group_SEdata)
}

final_random <- merge(random_group_data,random_group_SEdata, by=c("class","row"))
final_random <- merge(final_random,random_group_exposed, by=c("class"))
View(final_random)
write.csv(final_random,file=paste0(scoring_DolOcc_output,'DolOcc_random_effects.csv'), row.names = FALSE)

# create a txt file with the summary of the glmer model 
logsavefname <- "DolOcc_Final_Results.txt"
sink(paste0(final_DollOcc_modelling_output,logsavefname))
print(Sys.time()-dolocc.start.time)
print(summary(dolocc_final_model))
print(ranef(dolocc_final_model))
sink()

for(i in dolocc_random_campaigns){
  pdf(paste0(final_DollOcc_modelling_output,"DolOcc_Caterpillar_Plots ",i,".pdf"),width=7,height=5)
  print(ggCaterpillar(ranef(dolocc_final_model, condVar=TRUE)[i]))
  dev.off()
}

total_outliers_panid_to_remove <- unique(c(panid_out_all,occ_outliers_panid_to_remove,dolocc_outliers_panid_to_remove,pen_outliers_panid_to_remove))

# save all modeling data in .Rdata file
save(finaldata_dolocc,finaldata_pen,finaldata_occ,total_outliers_panid_to_remove,file=paste0(output,"saved_data/","Total_Modeling_data.Rdata"))

initial_data <- initial_data[!initial_data$panid %in% total_outliers_panid_to_remove,]
write.csv(initial_data,file=paste0(scoring_output,"final_data.csv"),row.names=F)

# create a summary for the outliers
logsavefname <- "Outliers Summary.txt"
sink(paste0(output,logsavefname))
print(paste0("Modeling Execution in: ",Sys.time()-Start.Time))
print(paste0("Number of outliers HH: ",length(unique(na.omit(total_outliers_panid_to_remove)))," from ",nrow(initial_data)))
print(cat("List of removed Experian IDs: "))
print(cat(na.omit(total_outliers_panid_to_remove)))
sink()

# calculate an estimation of total lift
total_lift_estimation <- ((1+coef(summary(pen_final_model))["group1","Estimate"])*(1+coef(summary(dolocc_final_model))["group1","Estimate"])*(1+coef(summary(occ_final_model))["group1","Estimate"])-1)
cat(paste0("last lift estimation: ",100*total_lift_estimation,"%"))













#Formula_final <- Trps_POS_P1 ~ Trps_PRE_P3 + Cpn_Trps_Shr_DPP_P3 + UN_per_Trip_PRE_P4 + UN_per_Trip_PRE_P5 + AP_Trps_Shr_DPP_P5 + AP_Trps_Shr_DPP_P6 + TPR_Trps_Shr_DPP_P8 + UN_per_Trip_PRE_P2 + Cpn_Trps_Shr_DPP_P1 + offset(log(Trps_PRE_P1 + 1)) + group+ (0 + group | estimated_hh_income) + (0 + group | hh_age) + (0 + group | number_of_children_in_living_Un) + (0 + group | Creative_Groups)

fo <- Trps_POS_P1 ~ Trps_PRE_P3 + UN_per_Trip_PRE_P4 + Cpn_Trps_Shr_DPP_P3 + UN_per_Trip_PRE_P5 + AP_Trps_Shr_DPP_P5 + AP_Trps_Shr_DPP_P6 + TPR_Trps_Shr_DPP_P8 + UN_per_Trip_PRE_P2 + Cpn_Trps_Shr_DPP_P1 + offset(log(Trps_PRE_P1 + 1)) + group+ (0 + group | estimated_hh_income) + (0 + group | hh_age) + (0 + group | number_of_children_in_living_Un) + (0 + group | Creative_Groups)

f <- "Trps_POS_P1 ~ Trps_PRE_P3 + UN_per_Trip_PRE_P4 + Trps_PRE_P4 + AP_Trps_Shr_DPP_P5 + UN_per_Trip_PRE_P6 + UN_per_Trip_PRE_P8 + Cpn_Trps_Shr_DPP_P1 + AP_Trps_Shr_DPP_P1 + offset(log(Trps_PRE_P1 + 1)) + group + (0 + group | estimated_hh_income) + (0 + group | hh_age) + (0 + group | number_of_children_in_living_Un) + (0 + group | Creative_Groups)"

# create nloptwrap optimization method
#defaultControl <- list(algorithm="NLOPT_LN_BOBYQA",xtol_rel=1e-6,maxeval=1e5)
#nloptwrap2 <- function(fn,par,lower,upper,control=list(),...) {
#  for (n in names(defaultControl)) 
#    if (is.null(control[[n]])) control[[n]] <- defaultControl[[n]]
#    res <- nloptr(x0=par,eval_f=fn,lb=lower,ub=upper,opts=control,...)
#    with(res,list(par=solution,
#                  fval=objective,
#                  feval=iterations,
#                  conv=if (status>0) 0 else status,
#                  message=message))
#}


new_occ_final_model <- glmer(f,data=finaldata_occ,family=Gamma(link="log"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))


occ_final_model <- glmer(Formula_final,data=finaldata_occ,family=Gamma(link="log"),control = glmerControl(calc.derivs = FALSE, optimizer="nloptwrap2"))


finaldata_occ$fitted <- fitted()
finaldata_occ$prop <- predict(occ_final_model,newdata=finaldata_occ,type="response", re.form = NA)
finaldata_occ$prop1 <- predict(occ_final_model,newdata=finaldata_occ,type="response",re.form = NULL)
finaldata_occ$summ <- 0.03965+finaldata_occ$Trps_PRE_P3*-0.028068+finaldata_occ$Cpn_Trps_Shr_DPP_P3*-0.133219+finaldata_occ$UN_per_Trip_PRE_P4*-0.02481+finaldata_occ$UN_per_Trip_PRE_P5*-0.009503+finaldata_occ$AP_Trps_Shr_DPP_P5*-0.098858+finaldata_occ$AP_Trps_Shr_DPP_P6*-0.112455+finaldata_occ$AP_Trps_Shr_DPP_P6*-0.112455+finaldata_occ$TPR_Trps_Shr_DPP_P8*-0.136959+finaldata_occ$UN_per_Trip_PRE_P2*-0.008434+finaldata_occ$Cpn_Trps_Shr_DPP_P1*0.162878+0.134898+log(Trps_PRE_P1 + 1)
x <- subset(finaldata_occ, panid =="2933862036")
x$summt <- exp(x$summ)
View(x)

Trps_POS_P1 ~ Trps_PRE_P3 + UN_per_Trip_PRE_P4 + Trps_PRE_P4 + AP_Trps_Shr_DPP_P5 + UN_per_Trip_PRE_P6 + UN_per_Trip_PRE_P8 + Cpn_Trps_Shr_DPP_P1 + AP_Trps_Shr_DPP_P1 + offset(log(Trps_PRE_P1 + 1)) + group 
+ (0 + group | estimated_hh_income) + (0 + group | hh_age) + (0 + group | number_of_children_in_living_Un) +      (0 + group | Creative_Groups)



x <- subset(finaldata_occ, panid =="2933862036")  
predict(occ_final_model,newdata=x,type="response", re.form = NA)


summary(finaldata_occ$Trps_POS_P1 ~ finaldata_occ$group)
summary(fitted(occ_final_model) ~ finaldata_occ$group)
summary(occ_final_model)
nrow(finaldata_occ)

finaldata_occ$prop <- predict(occ_final_model,newdata=finaldata_occ,type="response", re.form = NA)

summary(finaldata_occ$prop ~ finaldata_occ$group)

summary(fitted(dolocc_final_model) ~ finaldata_occ$group)
summary(finaldata_occ$Trps_POS_P1 ~ finaldata_occ$group)


x <- subset(finaldata_occ, panid =="2933862036")
predict(occ_final_model,newdata=x,type="response")
finaldata_occ$fitted <- fitted(occ_final_model)

save(finaldata_occ, file = "/media/u01/analytics/scoring/Healthy/modeling4/Scoring/occ.RData")
write.csv(initial_data,file=paste0(scoring_output,"final_data.csv"),row.names=F)
write.csv(finaldata_occ,file=paste0("/media/u01/analytics/scoring/Healthy/modeling4/Scoring/occ.csv"),row.names=F)

#---------------------- Vempali, SampathKumar -------------------------------
fixef(occ_final_model) + t(as.matrix(occ_final_model@pp$Zt)) * unlist(lme4::ranef(occ_final_model))
Zt <- getME(occ_final_model, "Zt")
X <- getME(occ_final_model, "X") 

######-----
zzzz <- occ_final_model@pp$Zt
fixef(occ_final_model) 
as.matrix(
           occ_final_model@pp$Zt
         )
* unlist(
          lme4::ranef(occ_final_model)
        )



x <- subset(finaldata_dolocc, panid =="2933862036")
predict(dolocc_final_model,newdata=x,type="response")
predict(dolocc_final_model,newdata=x,type="response", re.form = NA)
finaldata_dolocc$fitted <- fitted(dolocc_final_model)
summary(fitted(dolocc_final_model) ~ finaldata_dolocc$group)




summary(finaldata_occ$Trps_POS_P1 ~ finaldata_occ$group)
summary(fitted(occ_final_model) ~ finaldata_occ$group)
summary(occ_final_model)
nrow(finaldata_occ)

summary(finaldata_dolocc$Dol_per_Trip_POS_P1 ~ finaldata_dolocc$group)
summary(fitted(dolocc_final_model) ~ finaldata_dolocc$group)
summary(dolocc_final_model)
nrow(finaldata_dolocc)

summary(finaldata_pen$Buyer_Pos_P1 ~ finaldata_pen$group)
summary(fitted(pen_final_model) ~ finaldata_pen$group)
summary(pen_final_model)
nrow(finaldata_pen)

